In [760]:

import sys 
sys.path.append('..')

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from ctr.fm.model import FM as NEW_FM

from ctr.utils.data_process import create_criteo_dataset
import pandas as pd
import os
from sklearn.preprocessing import *

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 切换工作目录
os.chdir(r'C:\Users\Admin\Desktop\recommend-tf2.0-main\src\ctr')

In [2]:
pd.set_option('max_columns', 100)
# =============================== GPU ==============================
# gpu = tf.config.experimental.list_physical_devices(device_type='GPU')
# print(gpu)
os.environ['CUDA_VISIBLE_DEVICES'] = '2, 3'
# ========================= Hyper Parameters =======================
# you can modify your file path
file = '../data/criteo_sampled_data.csv'
read_part = True
sample_num = 100000
test_size = 0.2
df = pd.read_csv(file, nrows=4)
df.head()

def create_criteo_dataset(file, embed_dim=8, read_part=True, sample_num=100000, test_size=0.2):
    """
    a example about creating criteo dataset
    :param file: dataset's path
    :param embed_dim: the embedding dimension of sparse features
    :param read_part: whether to read part of it
    :param sample_num: the number of instances if read_part is True
    :param test_size: ratio of test dataset
    :return: feature columns, train, test
    """
    names = ['label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11',
             'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
             'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22',
             'C23', 'C24', 'C25', 'C26']

    if read_part:
        data_df = pd.read_csv(file, iterator=True)
        data_df = data_df.get_chunk(sample_num)

    else:
        data_df = pd.read_csv(file)

    sparse_features = ['C' + str(i) for i in range(1, 27)]
    dense_features = ['I' + str(i) for i in range(1, 14)]

    data_df[sparse_features] = data_df[sparse_features].fillna('-1')
    data_df[dense_features] = data_df[dense_features].fillna(0)

    for feat in sparse_features:
        le = LabelEncoder()
        data_df[feat] = le.fit_transform(data_df[feat].astype(str))

    # ==============Feature Engineering===================

    # dense_features = [feat for feat in data_df.columns if feat not in sparse_features + ['label']]


    for feat in dense_features:
        mms = MinMaxScaler()
        data_df[feat] = mms.fit_transform(data_df[dense_features].astype(int))

    feature_columns = [[denseFeature(feat) for feat in dense_features]] + \
                      [[sparseFeature(feat, len(data_df[feat].unique()), embed_dim=embed_dim)
                        for feat in sparse_features]]

    train, test = train_test_split(data_df, test_size=test_size)

    train_X = [train[dense_features].values.astype('float32'), train[sparse_features].values.astype('int32')]
    train_y = train['label'].values.astype('int32')
    test_X = [test[dense_features].values.astype('float32'), test[sparse_features].values.astype('int32')]
    test_y = test['label'] .values.astype('int32')

    return feature_columns, (train_X, train_y), (test_X, test_y)

# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(file=file,
                                       read_part=read_part,
                                       sample_num=sample_num,
                                       test_size=test_size)

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,1.0,2.0,NaN,2.0,68fd1e64,80e26c9b,fb936136,7b4723c4,25c83c98,7e0ccccf,de7995b8,1f89b562,a73ee510,a8cd5504,b2cb9c98,37c9c164,2824a5f6,1adce6ef,8ba8b39a,891b62e7,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,1.0,1.0,NaN,4.0,68fd1e64,f0cf0024,6f67f7e5,41274cd7,25c83c98,fe6b92e5,922afcc0,0b153874,a73ee510,2b53e5fb,4f1b46f3,623049e6,d7020589,b28479f6,e6c5b5cd,c92f3b61,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,1.0,3.0,3.0,45.0,287e684f,0a519c5c,02cf9876,c18be181,25c83c98,7e0ccccf,c78204a1,0b153874,a73ee510,3b08e48b,5f5e6091,8fe001f4,aa655a2f,07d13a8f,6dc710ed,36103458,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,68fd1e64,2c16a946,a9a87e68,2e17d6f6,25c83c98,fe6b92e5,2e8a689b,0b153874,a73ee510,efea433b,e51ddf94,a30567ca,3516f6e6,07d13a8f,18231224,52b8680f,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN


# FM

In [676]:
class FM(keras.Model):
    def __init__(self, latent_dim, feat_num):
        super().__init__()
        # 隐向量
        self.v = self.add_weight(shape=(feat_num, latent_dim),
                                initializer=keras.initializers.glorot_normal(),
                                trainable=True)
        self.w = self.add_weight(shape=(feat_num, 1),
                                initializer=keras.initializers.glorot_normal()
                                ,trainable=True)
        self.bias = tf.Variable([0,], dtype=tf.float32, trainable=True)
    
    def call(self, x):
        # x shape [batch, feat_num]
        # 一阶交叉
        first_order = self.bias + tf.matmul(x, self.w) # [batch, 1]
        # 二阶交叉
        second_order = 0.5*(tf.reduce_sum(
            tf.pow(tf.matmul(x, self.v), 2) # [batch, latent] 
            - tf.matmul(tf.pow(x, 2), tf.pow(self.v, 2)),
                axis=1,
            keepdims=True
        ))
        return first_order + second_order


In [ ]:
tf.keras.backend.clear_session()
train_X, train_y = train
test_X, test_y = test
k = 8
train_x = np.concatenate(train_X, axis=1)
test_x = np.concatenate(test_X, axis=1)
learning_rate = 0.001
batch_size = 512
epochs = 5



model = FM(k, train_x.shape[1])
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = keras.losses.binary_crossentropy
metric = tf.keras.metrics.AUC()

# 创建训练集
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_data = train_data.repeat(3).batch(batch_size)


for i in range(1, epochs+1):
    for idx, (train_x1, train_y1) in enumerate(train_data):
#         print(train_x1.shape, train_y1.shape)
        with tf.GradientTape() as tape:
            y_hat = model(train_x1)
            loss = loss_fn(train_y1, tf.nn.sigmoid(y_hat))
            gradient = tape.gradient(loss, model.trainable_variables)
            opt.apply_gradients(zip(gradient, model.trainable_variables)) # 需要zip一下配对
        if idx % 300 == 0:
            print(f'epoch={i}, batch={idx} loss={tf.reduce_mean(loss)}')
    auc = metric(test_y,  tf.nn.sigmoid(model(test_x)))
    print(f'epoch={i}, auc={auc}')

# 调包实现fm

In [684]:
tf.keras.backend.clear_session()
train_X, train_y = train
test_X, test_y = test

k = 8

learning_rate = 0.001
batch_size = 512
epochs = 10
# ============================model checkpoint======================
check_path = './save/fm_weight.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
                                                verbose=1, save_best_only=True, monitor='val_auc',
                                               mode='max')

# mirrored_strategy = tf.distribute.MirroredStrategy()
# with mirrored_strategy.scope():
fm = NEW_FM(feature_columns=feature_columns, k=k) # 包里的实现
model = fm.build_graph()
if os.path.exists(check_path + '.index'):
    print('加载模型权重参数！')
    model.load_weights(check_path)

#     model.summary()
# ============================Compile============================
model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate),
              metrics=[tf.keras.metrics.AUC()])


# ==============================Fit==============================
model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),
              checkpoint],  # checkpoint
    batch_size=batch_size,
    validation_split=0.1
)

# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])


加载模型权重参数！
Epoch 1/10
141/141 [==============================] - ETA: 0s - loss: 0.7615 - auc: 0.4763
Epoch 00001: val_auc improved from -inf to 0.46974, saving model to ./save\fm_weight.ckpt
141/141 [==============================] - 69s 484ms/step - loss: 0.7615 - auc: 0.4763 - val_loss: 0.7633 - val_auc: 0.4697
Epoch 2/10
141/141 [==============================] - ETA: 0s - loss: 0.7615 - auc: 0.4763
Epoch 00002: val_auc did not improve from 0.46974
141/141 [==============================] - 74s 524ms/step - loss: 0.7615 - auc: 0.4763 - val_loss: 0.7633 - val_auc: 0.4697
Epoch 3/10
141/141 [==============================] - ETA: 0s - loss: 0.7615 - auc: 0.4763
Epoch 00003: val_auc did not improve from 0.46974
40/40 [==============================] - 21s 526ms/step - loss: 0.7596 - auc: 0.4874
test AUC: 0.487398


# GBDT + LR

In [40]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

gb = GradientBoostingClassifier(n_estimators=50, random_state=10, subsample=.6, max_depth=7,
                               min_samples_split=500)
gb.fit(np.concatenate(train_X, axis=1), train_y)


GradientBoostingClassifier(max_depth=7, min_samples_split=500, n_estimators=50,
                           random_state=10, subsample=0.6)

In [52]:
new_feature = gb.apply(np.concatenate(train_X, axis=1)).reshape(-1, 50) # apply 方法放回在 50棵树中，每个训练样本落的叶子节点的索引
new_feature.shape

(80000, 50)

In [53]:
enc = OneHotEncoder(sparse=False).fit(new_feature)
train_new_x = enc.transform(new_feature)
test_new_x  = enc.transform(gb.apply(np.concatenate(test_X, axis=1)).reshape(-1, 50))

In [ ]:
from sklearn.metrics import roc_auc_score

lr = LogisticRegression(C=0.5, max_iter=500)
lr.fit(train_new_x, train_y)
       
print('在LR部分只使用GBDT得到的组合特征')
print('训练集：auc= %.4f' % roc_auc_score(train_y, lr.predict_proba(train_new_x)[:, 1]))
print('测试集：auc= %.4f' % roc_auc_score(test_y, lr.predict_proba(test_new_x)[:, 1]))

In [ ]:
lr = LogisticRegression(C=0.5, max_iter=500)
lr.fit(np.concatenate((train_new_x, np.concatenate(train_X, axis=1)), axis=1), train_y)

print('在LR部分使用GBDT得到的组合特征 + 原始特征')
print('训练集：auc= %.4f' % roc_auc_score(train_y, lr.predict_proba(np.concatenate((train_new_x, np.concatenate(train_X, axis=1)), axis=1))[:, 1]))
print('测试集：auc= %.4f' % roc_auc_score(test_y, lr.predict_proba(np.concatenate((test_new_x, np.concatenate(test_X, axis=1)), axis=1))[:, 1]))

### 分析
从结果来看，似乎在LR部分不应该再加入原始特征部分

In [74]:
import gc
gc.collect()

4302

# MLR

In [75]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [148]:
#数据处理
def data_concat(train,test):
    train['type'] = 1
    test['type'] = 2

    
    all_columns=['age','workclass','fnlwgt','education','education-num','marital-status',
                'occupation','relationship','race','sex','capital-gain','capital-loss',
                'hours-per-week','native-country','label','type']
    all_data=pd.concat([train,test],axis=0)
    all_data.columns=all_columns
    return all_data

def data_processing(train,test):
    df=data_concat(train,test)
    continus_columns=['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']
    category_columns=['workclass','education','marital-status','occupation','relationship','race','sex','native-country']
    #类别变量做one_hot_encoding
    df=pd.get_dummies(df,columns=category_columns)
    #连续数据标准化
    for col in continus_columns:
        ss=StandardScaler()
        df[col]=ss.fit_transform(df[[col]])

    df['label']=df['label'].apply(lambda x: 1 if  x.strip()=='>50K'  else 0)

    return df


print(os.getcwd())
train_data=pd.read_table(r'../data/adult.data',header=None,delimiter=',')
test_data=pd.read_table(r'../data/adult.test',header=None,delimiter=',', skiprows=[0])
test_data[14]=test_data[14].apply(lambda x: x[:-1])
df = data_processing(train_data,test_data)
train_data=df[df['type']==1].drop(['type', 'label'],axis=1).astype(np.float32) 
train_label = df.loc[df['type']==1, ['label']].astype(np.float32) 
test_data=df[df['type']==2].drop(['type', 'label'],axis=1).astype(np.float32) 
test_label = df.loc[df['type']==2, ['label']].astype(np.float32) 

C:\Users\Admin\Desktop\recommend-tf2.0-main\src\ctr


### 建模
1. 使用用户数据进行聚类（实际就是一个m输出问题，activation = softmax）
2. 使用广告数据进行分类训练逻辑回归模型（实际就是一个m输出问题， activation= sigmoid）

In [178]:
import tensorflow as tf
import tensorflow.keras as keras


keras.backend.clear_session()

In [203]:
m = 12
x = keras.Input(shape=(108,))
w = keras.layers.Dense(m, activation='softmax', kernel_regularizer=keras.regularizers.l1_l2(0.001))(x)
u = keras.layers.Dense(m, activation='sigmoid', kernel_regularizer=keras.regularizers.l1_l2(0.001))(x)
output = tf.clip_by_value(tf.math.reduce_sum(tf.multiply(w, u), axis=1, keepdims=True),  0.00001, 1 - 0.00001)

model = tf.keras.Model(inputs=x, outputs=output)

In [206]:
def my_loss(ytrue, y_pred):
    val = tf.add(tf.multiply(ytrue, tf.math.log(y_pred)),
           tf.multiply(1-ytrue, tf.math.log(1-y_pred)))
    return -tf.reduce_sum(val) # 损失函数是负对数似然

model.compile(loss=my_loss,
             optimizer='adam',
             metrics=[keras.metrics.AUC()])


In [207]:
model.summary()
model.fit(train_data, train_label, epochs=10, batch_size=32)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 108)]        0           []                               
                                                                                                  
 dense_12 (Dense)               (None, 12)           1308        ['input_7[0][0]']                
                                                                                                  
 dense_13 (Dense)               (None, 12)           1308        ['input_7[0][0]']                
                                                                                                  
 tf.math.multiply_6 (TFOpLambda  (None, 12)          0           ['dense_12[0][0]',               
 )                                                                'dense_13[0][0]']         

In [211]:
print('测试集合', roc_auc_score(test_label.squeeze(), model.predict(test_data).squeeze()))

测试集合 0.9106644201433518


# NeuralCF

In [212]:
'''
Created on Aug 8, 2016
Processing datasets. 

@author: Xiangnan He (xiangnanhe@gmail.com)
'''
import scipy.sparse as sp
import numpy as np

class Dataset(object):
    '''
    classdocs
    '''

    def __init__(self, path):
        '''
        Constructor
        '''
        self.trainMatrix = self.load_rating_file_as_matrix(path + ".train.rating")
        self.testRatings = self.load_rating_file_as_list(path + ".test.rating")
        self.testNegatives = self.load_negative_file(path + ".test.negative")
        assert len(self.testRatings) == len(self.testNegatives)
        
        self.num_users, self.num_items = self.trainMatrix.shape
        
    def load_rating_file_as_list(self, filename):
        ratingList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item = int(arr[0]), int(arr[1])
                ratingList.append([user, item])
                line = f.readline()
        return ratingList
    
    def load_negative_file(self, filename):
        negativeList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                negatives = []
                for x in arr[1: ]:
                    negatives.append(int(x))
                negativeList.append(negatives)
                line = f.readline()
        return negativeList
    
    def load_rating_file_as_matrix(self, filename):
        '''
        Read .rating file and Return dok matrix.
        The first line of .rating file is: num_users\t num_items
        '''
        # Get number of users and items
        num_users, num_items = 0, 0
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                u, i = int(arr[0]), int(arr[1])
                num_users = max(num_users, u)
                num_items = max(num_items, i)
                line = f.readline()
        # Construct matrix
        mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
                if (rating > 0):
                    mat[user, item] = 1.0
                line = f.readline()    
        return mat


In [217]:
dataset = Dataset('NeuralCF-master/Data/' + 'ml-1m')
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives

In [262]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    k = 0
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(numItems)
            while (u, j) in train: #train.has_key((u, j)):
                j = np.random.randint(numItems)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
        if len(user_input) > 1e5: break # 小规模进行训练
        
    return user_input, item_input, labels   

user_input, item_input, labels = get_train_instances(train, 4)

## GMF

In [287]:
numUser, numItems = train.shape
latent_dim = 20
feat_user = keras.Input(shape=(1,))
embedding_user = keras.layers.Embedding(input_dim=numUser, output_dim=latent_dim)(feat_user)

feat_item = keras.Input(shape=(1,))
embedding_item = keras.layers.Embedding(input_dim=numItems, output_dim=latent_dim)(feat_item)
# new_feat = tf.concat([embedding_user, embedding_item])
# 可以通过Flatten展平embedding向量
embedding_item = keras.layers.Flatten()(embedding_item)
embedding_user = keras.layers.Flatten()(embedding_user)

out_put = tf.multiply(embedding_user, embedding_item)
out = tf.keras.layers.Dense(1, activation='sigmoid')(out_put)

model = tf.keras.Model(inputs=[feat_user, feat_item], outputs=out)
model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.AUC()]
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 1, 20)        120800      ['input_3[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, 1, 20)        74120       ['input_4[0][0]']                
                                                                                            

In [288]:
model.fit([np.array(user_input), np.array(item_input)], np.array(labels)[:,np.newaxis], epochs=5, batch_size=32)

Epoch 1/5
3126/3126 [==============================] - 4s 1ms/step - loss: 0.4600 - auc_1: 0.7127
Epoch 2/5
3126/3126 [==============================] - 4s 1ms/step - loss: 0.3313 - auc_1: 0.8784
Epoch 3/5
3126/3126 [==============================] - 4s 1ms/step - loss: 0.2822 - auc_1: 0.9169
Epoch 4/5
3126/3126 [==============================] - 4s 1ms/step - loss: 0.2347 - auc_1: 0.9456
Epoch 5/5
3126/3126 [==============================] - 4s 1ms/step - loss: 0.1883 - auc_1: 0.9669


## MLP

In [289]:
numUser, numItems = train.shape
latent_dim = 20
feat_user = keras.Input(shape=(1,))
embedding_user = keras.layers.Embedding(input_dim=numUser, output_dim=latent_dim)(feat_user)
embedding_user = tf.keras.layers.Flatten()(embedding_user)

feat_item = keras.Input(shape=(1,))
embedding_item = keras.layers.Embedding(input_dim=numItems, output_dim=latent_dim)(feat_item)
embedding_item = tf.keras.layers.Flatten()(embedding_item)

x = tf.concat([embedding_user, embedding_item], axis=1)

hidden_units = [128, 64, 32]
for i in hidden_units:
    layer = keras.layers.Dense(i, activation='relu', kernel_regularizer=keras.regularizers.l2())
    x = layer(x)

outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model2 = keras.Model(inputs=[feat_user, feat_item], outputs=outputs)

In [290]:
keras.backend.clear_session()
model2.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(), metrics=[keras.metrics.AUC()])
model2.fit([np.array(user_input), np.array(item_input)], np.array(labels)[:,np.newaxis], epochs=5, batch_size=32)

Epoch 1/5
3126/3126 [==============================] - 6s 2ms/step - loss: 0.4827 - auc: 0.7793
Epoch 2/5
3126/3126 [==============================] - 5s 2ms/step - loss: 0.3763 - auc: 0.8614
Epoch 3/5
3126/3126 [==============================] - 5s 2ms/step - loss: 0.3627 - auc: 0.8663
Epoch 4/5
3126/3126 [==============================] - 5s 2ms/step - loss: 0.3561 - auc: 0.8683
Epoch 5/5
3126/3126 [==============================] - 5s 2ms/step - loss: 0.3522 - auc: 0.8694


## GMF + MLP 
注意 GMF 与 MLP 的隐向量需要各自独立训练

In [299]:
keras.backend.clear_session()

numUser, numItems = train.shape

feat_user = keras.Input(shape=(1,))
feat_item = keras.Input(shape=(1,))
# gmf embedding
latent_dim = 20
gmf_embedding_user = keras.layers.Embedding(input_dim=numUser, output_dim=latent_dim)(feat_user)
gmf_embedding_user = tf.keras.layers.Flatten()(gmf_embedding_user)


gmf_embedding_item = keras.layers.Embedding(input_dim=numItems, output_dim=latent_dim)(feat_item)
gmf_embedding_item = tf.keras.layers.Flatten()(gmf_embedding_item)

# # mlp embedding
latent_dim = 30
mlp_embedding_user = keras.layers.Embedding(input_dim=numUser, output_dim=latent_dim)(feat_user)
mlp_embedding_user = tf.keras.layers.Flatten()(mlp_embedding_user)


mlp_embedding_item = keras.layers.Embedding(input_dim=numItems, output_dim=latent_dim)(feat_item)
mlp_embedding_item = tf.keras.layers.Flatten()(mlp_embedding_item)


# GMF部分·
y1 = tf.multiply(gmf_embedding_item, gmf_embedding_user)
# MLP部分
x = tf.concat([mlp_embedding_user, mlp_embedding_item], axis=1)
hidden_units = [128, 64, 32]
for i in hidden_units:
    layer = keras.layers.Dense(i, activation='relu', kernel_regularizer=keras.regularizers.l2())
    x = layer(x)

x = tf.concat([y1, x], axis=1)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=[feat_user, feat_item], outputs=outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 1, 30)        181200      ['input_1[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, 1, 30)        111180      ['input_2[0][0]']                
                                                                                              

In [300]:
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(), metrics=[keras.metrics.AUC()])
model.fit([np.array(user_input), np.array(item_input)], np.array(labels)[:,np.newaxis], epochs=5, batch_size=32)

Epoch 1/5
3126/3126 [==============================] - 9s 3ms/step - loss: 0.4778 - auc: 0.7797
Epoch 2/5
3126/3126 [==============================] - 8s 3ms/step - loss: 0.3222 - auc: 0.8965
Epoch 3/5
3126/3126 [==============================] - 8s 3ms/step - loss: 0.2473 - auc: 0.9394
Epoch 4/5
3126/3126 [==============================] - 9s 3ms/step - loss: 0.1937 - auc: 0.9650
Epoch 5/5
3126/3126 [==============================] - 8s 3ms/step - loss: 0.1504 - auc: 0.9799


# PNN

In [411]:
pd.set_option('max_columns', 100)
keras.backend.clear_session()
# =============================== GPU ==============================
# gpu = tf.config.experimental.list_physical_devices(device_type='GPU')
# print(gpu)
os.environ['CUDA_VISIBLE_DEVICES'] = '2, 3'
# ========================= Hyper Parameters =======================
# you can modify your file path
file = '../data/criteo_sampled_data.csv'
read_part = True
sample_num = 100000
test_size = 0.2
embed_dims = 8 # embedding 的维度
df = pd.read_csv(file, nrows=4)
df.head()
# ========================== Create dataset =======================
# train的结构是元组 ([dense_feature, sparse_feature], label_)

feature_columns, train, test = create_criteo_dataset(file=file,
                                       read_part=read_part,
                                       sample_num=sample_num,
                                       test_size=test_size,
                                        embed_dim=embed_dims)

In [418]:
def dnn(units_in_prdoucts = 256): # dnn 结构，参数为输入dnn的特征维度
    return keras.Sequential(
    [
        keras.Input(shape=(units_in_prdoucts,)),
        keras.layers.Dropout(0.3),
    *[ keras.layers.Dense(item, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)) for item in (128, 64, 32)]
    , keras.layers.Dropout(0.2)
    ,keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.01)) # for ctr
    ]
    )


In [413]:
dense_feat = keras.Input(shape=(13,))
sparse_feat = keras.Input(shape=(26,))

res = []
for i, dic_ in enumerate(feature_columns[1]):
    kinds = dic_['feat_num']
    k = dic_['embed_dim']
    emb_layer = keras.layers.Embedding(kinds, k)
    res.append(tf.expand_dims(emb_layer(sparse_feat[:, i]), axis=1)) # 将 axis=1 设置为 filed域
sparse_embeds = tf.concat(res, axis=1) # [batch, fileds, embed_num]

In [416]:
# Product 层
mode = 'out' 
p_d = 35 # 设定 product层的输出维度
embed_dims = 8


# bias维度
l_b = tf.Variable([[0]*p_d], dtype=tf.float32) # 注意, 必须使用两维来表示，使用一维无法与 l_z 和 l_p 进行 add
# l_z 部分
x = tf.keras.layers.Flatten()(sparse_embeds)
l_z = tf.keras.layers.Dense(p_d, use_bias=False, kernel_regularizer=keras.regularizers.l2(.01))(x)  #[batch, p_d]
# l_p 部分
# tf.matmul 三维张量里面每一个对应矩阵做矩阵乘法
if mode == 'in':
    # 得到内积矩阵
    p = tf.matmul(sparse_embeds, tf.transpose(sparse_embeds, perm=[0, 2, 1])) # [batch, fileds, fileds]
else:
    # 外积
    # 得到外积矩阵,采用降维算法 先池化所有的样本的特征域得到一个embedding
    tmp = tf.expand_dims(tf.reduce_sum(sparse_embeds, axis=1), axis=1) #[batch ,1, embed_dim]
    p = tf.matmul(tf.transpose(tmp, perm=[0, 2, 1]), tmp) #[batch, embed_dim, embed_dim]
    

class my_product(keras.layers.Layer):
    def __init__(self, p_d, mode='in', embed_dims=embed_dims):
        self.p_d = p_d
        self.mode = mode
        self.embed_dims = embed_dims
        super().__init__()
    
    def build(self, input_shape):
        if self.mode == 'in':
            # 对于每个product的输出节点都有一个 fileds * fileds 维的权重矩阵
            self.w = self.add_weight(
                shape=(input_shape[-1], input_shape[-1], self.p_d),
                initializer=keras.initializers.random_normal(),
                trainable=True
            ) # [fileds, fileds, p_d]
        else:
            self.w = self.add_weight(
            shape=(self.embed_dims, self.embed_dims, self.p_d),
            initializer=keras.initializers.random_normal(),
                trainable=True  
            ) # [embed_dim, embed_dim, p_d]
            
        super().build(input_shape) # 调用父类方法
        
    def call(self, inputs):
        # axes 指出使用 inputs的axis=0维度（不使用1，2），self.w axis=2维度（不使用0，1）做矩阵乘法，得到 shape=(dim_inputs_0, dim_self.w_2).
        # 主要要求 inputs的axis(1, 2) 要和 self.w 的 axis(0, 1) 一致，实际上做的是矩阵内积和
        return tf.tensordot(inputs, self.w, axes=[(1, 2), (0, 1)]) 

l_p = my_product(p_d, mode)(p)
# product的l1层 l1 层做加法
l1 = keras.layers.Add()([l_z, l_p, l_b])

out = tf.concat([l1, dense_feat], axis=-1) # 与连续特征拼接
outputs = dnn(out.shape[-1])(out)
# l1.out_puts
model = keras.Model(inputs=[dense_feat, sparse_feat], outputs=outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 26)]         0           []                               
                                                                                                  
 tf.__operators__.getitem (Slic  (None,)             0           ['input_2[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 tf.__operators__.getitem_1 (Sl  (None,)             0           ['input_2[0][0]']                
 icingOpLambda)                                                                                   
                                                                                            

In [417]:
model.compile(loss=tf.keras.losses.binary_crossentropy,
             optimizer=keras.optimizers.Adam(),
             metrics=[keras.metrics.AUC()])

model.fit(train[0], train[1], epochs=5, batch_size=32, validation_data=test)

Epoch 1/5
2500/2500 [==============================] - 39s 14ms/step - loss: 0.3439 - auc_1: 0.9479 - val_loss: 0.6654 - val_auc_1: 0.6865
Epoch 2/5
2500/2500 [==============================] - 36s 14ms/step - loss: 0.2323 - auc_1: 0.9571 - val_loss: 0.6787 - val_auc_1: 0.6864
Epoch 3/5
2500/2500 [==============================] - 35s 14ms/step - loss: 0.2170 - auc_1: 0.9614 - val_loss: 0.7147 - val_auc_1: 0.6809
Epoch 4/5
2500/2500 [==============================] - 35s 14ms/step - loss: 0.2053 - auc_1: 0.9650 - val_loss: 0.7505 - val_auc_1: 0.6754
Epoch 5/5
2500/2500 [==============================] - 35s 14ms/step - loss: 0.1971 - auc_1: 0.9673 - val_loss: 0.7675 - val_auc_1: 0.6721


# wide & deep

In [495]:
# 准备数据
pd.set_option('max_columns', 100)
keras.backend.clear_session()
# =============================== GPU ==============================
# gpu = tf.config.experimental.list_physical_devices(device_type='GPU')
# print(gpu)
os.environ['CUDA_VISIBLE_DEVICES'] = '2, 3'
# ========================= Hyper Parameters =======================
# you can modify your file path
file = '../data/criteo_sampled_data.csv'
read_part = True
sample_num = 100000
test_size = 0.2
embed_dims = 8 # embedding 的维度
df = pd.read_csv(file, nrows=4)
df.head()
# ========================== Create dataset =======================
# train的结构是元组 ([dense_feature, sparse_feature], label_)

feature_columns, train, test = create_criteo_dataset(file=file,
                                       read_part=read_part,
                                       sample_num=sample_num,
                                       test_size=test_size,
                                        embed_dim=embed_dims)

In [562]:
class Linear(keras.layers.Layer):
    """
    将输出线性组合为一个输出单元，不需要经过激活函数处理
    """
    def __init__(self, *args, reg_l1=0.01, **kwgs):
        super().__init__(*args, **kwgs)
        self.linear = keras.layers.Dense(1, kernel_regularizer=keras.regularizers.l2(reg_l1), activation=None) # 使用正则化可以防止过拟合
    
    def call(self, inputs):
        return self.linear(inputs)


class DNN(keras.layers.Layer):
    def __init__(self, hidden_units, activation='relu', dropout=0, reg_l2=0.01,**kwgs):
        super().__init__(**kwgs)
        self.dnn_network = [keras.layers.Dense(i, activation=activation, 
                                               kernel_regularizer=keras.regularizers.l2(reg_l2),  
                                               bias_regularizer=keras.regularizers.l2(reg_l2)) for i in hidden_units] # 使用正则化可以防止过拟合
        self.dropout = keras.layers.Dropout(dropout)
    
    def call(self, inputs):
        x = inputs
        for layer in self.dnn_network:
            x = layer(x)
        x = self.dropout(x)
        return x

    
class WideDeep(keras.Model):
    def __init__(self, feature_column, hidden_units, activation='relu', dnn_dropout=0, embed_reg=1e-4, **kwgs):
        super().__init__(**kwgs)
        dense_feature_array, sparse_feature_array = feature_column
        self.dense_dim, self.sparse_dim = len(dense_feature_array), len(sparse_feature_array) # 特征维度
        # embed_map
        self.sparse_embed_map = {
            'embed_' + str(idx): keras.layers.Embedding(
                                         input_dim=feat['feat_num'],
                                         input_length=1, # 指名进行 embeding 的序列长度
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=keras.regularizers.l2(embed_reg)
            ) for idx, feat in enumerate(sparse_feature_array)
        }
        self.wide = Linear() # 线性部分 wide 部分
        self.dnn = DNN(hidden_units, activation=activation, dropout=dnn_dropout, **kwgs)
        self.final_deep = keras.layers.Dense(1, activation=None) # deep 部分的输出综合为一个节点
        
    def call(self, inputs):
        dense_feat, sparse_feat = inputs
        # embed category
        res = []
        for i in range(sparse_feat.shape[1]):
            embed_layer = self.sparse_embed_map['embed_'+str(i)]
            res.append(embed_layer(sparse_feat[:,i])) # 输入embed的维度是 batch, sepquence_dim
        # dnn 部分
        embed_cat = tf.concat(res, axis=-1)
        dnn_input = tf.concat([embed_cat, dense_feat], axis=-1)
        deep_out = self.dnn(dnn_input) # 多层MLP的输出
        deep_out = self.final_deep(deep_out)
        
        # wide 部分
        wide_out = self.wide(dense_feat)
        
        # 综合方式
        out = 0.5*(wide_out + deep_out)
        return tf.nn.sigmoid(out)
    
    
    def build_graph(self): # 构建静态图，放回模型
        dense_inputs = keras.Input(shape=(self.dense_dim,), dtype=tf.float32)
        sparse_inputs = keras.Input(shape=(self.sparse_dim,), dtype=tf.float32)
        model = keras.Model(inputs=[dense_inputs, sparse_inputs],
                           outputs=self.call([dense_inputs, sparse_inputs]))
        return model
        

In [563]:
keras.backend.clear_session()
sample_num = 100000
test_size = 0.2

embed_dim = 8
dnn_dropout = 0.5
hidden_units = [256, 128, 64]

learning_rate = 0.001
batch_size = 512
epochs = 10
    
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
# mirrored_strategy = tf.distribute.MirroredStrategy() # 分布式训练策略
# with mirrored_strategy.scope():
wdl = WideDeep(feature_columns, hidden_units=hidden_units, dnn_dropout=dnn_dropout)
model = wdl.build_graph()
# ============================Compile============================
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=[keras.metrics.AUC()])


model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],  # checkpoint
    batch_size=batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])


Epoch 1/10
141/141 [==============================] - 5s 21ms/step - loss: 1.9065 - auc: 0.5211 - val_loss: 0.6531 - val_auc: 0.6851
Epoch 2/10
141/141 [==============================] - 3s 19ms/step - loss: 0.5836 - auc: 0.6294 - val_loss: 0.5544 - val_auc: 0.6964
Epoch 3/10
141/141 [==============================] - 3s 18ms/step - loss: 0.5481 - auc: 0.6845 - val_loss: 0.5405 - val_auc: 0.7081
Epoch 4/10
141/141 [==============================] - 3s 19ms/step - loss: 0.5379 - auc: 0.7073 - val_loss: 0.5352 - val_auc: 0.7185
Epoch 5/10
141/141 [==============================] - 3s 19ms/step - loss: 0.5324 - auc: 0.7198 - val_loss: 0.5301 - val_auc: 0.7262
Epoch 6/10
141/141 [==============================] - 3s 19ms/step - loss: 0.5283 - auc: 0.7289 - val_loss: 0.5286 - val_auc: 0.7294
Epoch 7/10
141/141 [==============================] - 3s 19ms/step - loss: 0.5236 - auc: 0.7417 - val_loss: 0.5284 - val_auc: 0.7339
Epoch 8/10
141/141 [==============================] - 3s 19ms/step - 

### 不规范的写法

In [587]:
keras.backend.clear_session()
# wide 部分
inputs_wide = keras.Input(shape=(len(feature_columns[0]),)) # 目前假设以dense 特征作为 wide 部分，实际上wide部分需要较多的人工干预进行选择
# 一般会使用 l1 正则控制 wide 部分的参数的稀疏性
out_wide = keras.layers.Dense(1, activation=None)(inputs_wide) # Wide 部分的输出不需要经过激活函数

wide = keras.Model(inputs=inputs_wide, outputs=out_wide)

In [588]:

# deep 特征提取部分，输出为一个值
def dnn(units_in_prdoucts = 256): 
    return keras.Sequential(
    [
        keras.Input(shape=(units_in_prdoucts,)),
    *[ keras.layers.Dense(item, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)) for item in (256, 128, 64)]
    ,keras.layers.Dropout(0.1)
    ,keras.layers.Dense(1, activation=None) # Deep 部分的输出不需要经过激活函数
    ]
    )

# deep 部分
inputs_deep = keras.Input(shape=(len(feature_columns[1]),))
# embeding for categories
res = []
for i, dic_ in enumerate(feature_columns[1]):
    embed = keras.layers.Embedding(dic_['feat_num'], embed_dims)
    res.append(embed(inputs_deep[:, i]))
# concat embedding vector
dense_embed = tf.concat(res, axis=-1)
deep_feat = tf.concat([dense_embed, inputs_wide], axis=-1) # 拼接deep的输入， embedding + dense_feature
input_shape = dense_embed.shape[-1]
dnn_network = dnn(input_shape)
out = dnn_network(dense_embed)

deep = keras.Model(inputs=inputs_deep, outputs=out)

In [560]:

final_layer = keras.layers.Dense(1, use_bias=False, activation='sigmoid') # 细节部分不适用bias
out_finall = final_layer(0.5*(deep(inputs_deep)+wide(inputs_wide)))
wide_deep = keras.Model(inputs=[inputs_wide,inputs_deep], outputs=out_finall) # 函数 api 建立模型

In [561]:
model = wide_deep
# model.summary()
model.compile(loss=keras.losses.binary_crossentropy,
                 optimizer=keras.optimizers.Adam(learning_rate=0.001),
                 metrics=[keras.metrics.AUC()])

model.fit(train[0], train[1], epochs=15, batch_size=256, validation_data=test)

Epoch 1/15
313/313 [==============================] - 8s 19ms/step - loss: 1.1338 - auc: 0.5997 - val_loss: 0.5225 - val_auc: 0.7037
Epoch 2/15
313/313 [==============================] - 6s 18ms/step - loss: 0.4945 - auc: 0.7777 - val_loss: 0.5075 - val_auc: 0.7307
Epoch 3/15
313/313 [==============================] - 6s 18ms/step - loss: 0.4127 - auc: 0.8870 - val_loss: 0.5484 - val_auc: 0.7090
Epoch 4/15
313/313 [==============================] - 6s 19ms/step - loss: 0.3360 - auc: 0.9340 - val_loss: 0.5966 - val_auc: 0.7004
Epoch 5/15
313/313 [==============================] - 6s 19ms/step - loss: 0.2894 - auc: 0.9513 - val_loss: 0.6037 - val_auc: 0.6988
Epoch 6/15
313/313 [==============================] - 6s 20ms/step - loss: 0.2607 - auc: 0.9599 - val_loss: 0.6416 - val_auc: 0.6909
Epoch 7/15
313/313 [==============================] - 6s 20ms/step - loss: 0.2410 - auc: 0.9652 - val_loss: 0.6476 - val_auc: 0.6817
Epoch 8/15
313/313 [==============================] - 7s 21ms/step - 

# Deep&Cross

In [634]:
class DNN(keras.layers.Layer):
    def __init__(self, hidden_units, activation='relu', dropout=0.1, reg_l2=0.01,**kwgs):
        super().__init__(**kwgs)
        self.dnn_network = [keras.layers.Dense(i, activation=activation, 
                                               kernel_regularizer=keras.regularizers.l2(reg_l2),  
                                               bias_regularizer=keras.regularizers.l2(reg_l2)) for i in hidden_units] # 使用正则化可以防止过拟合
        self.dropout = keras.layers.Dropout(dropout)
    
    def call(self, inputs):
        x = inputs
        for layer in self.dnn_network:
            x = layer(x)
        x = self.dropout(x)
        return x

class Cross(keras.layers.Layer):
    def __init__(self, layer_nums, input_length):
        """
        layer_nums : cross network的层数
        input_length: cross network的输入维度 
        """
        super().__init__()
        self.layer_nums = layer_nums
        self.units = input_length
    
    
    def build(self, input_shape):
        """设置参数共享变量"""
        self.w = {}
        self.b = {}
        for i in range(self.layer_nums):
            w, b = 'w_' + str(i), 'b_' + str(i) 
            self.w[w] = self.add_weight(shape=(self.units, 1), # 维度为 [n, 1]
                                             initializer=keras.initializers.glorot_normal(),
                                             trainable=True)
            self.b[b] = self.add_weight(shape=(self.units, 1),
                                             initializer=keras.initializers.glorot_normal(),
                                             trainable=True)
        super().build(input_shape) # 调用父类方法
        
        
    def call(self, inputs):
        x = inputs #[batch, n]
        x = tf.expand_dims(x, axis=-1) # [batch, n, 1]
        for i in range(self.layer_nums):
            w, b = 'w_' + str(i), 'b_' + str(i)
            pre = tf.identity(x) # 复制 x 一份
            x_t = tf.transpose(x, perm=[0, 2, 1]) # [batch, 1, n]
            x = tf.matmul(x, x_t) # [batch, n, n]
            x = tf.matmul(x, self.w[w]) # [batch, n, 1] # self.w[w] 是二维的，但是matmul支持广播运行
            x += self.b[b] + pre # [batch, n, 1]
            
#         print(x.shape, type(tf.squeeze(x)), tf.squeeze(x).shape)
        # 需要注意的是 使用 squeeze 指名将最后一个维度去除， 如果不指名具体的axis的话，最后squeeze得到的shape为 unknow, 主要是因为 shape 里面存在 None 表示不定长导致的
        # 所以当在遇见 shape 里面含有 None 的时候，降维需要指定具体的axis
        return tf.squeeze(x, axis=-1) # [batch, n]
        

class CrossDeep(keras.Model):
    def __init__(self, feature_column, hidden_units, cross_layers=2, activation='relu', dnn_dropout=0, embed_reg=1e-4, **kwgs):
        super().__init__(**kwgs)
        dense_feature_array, sparse_feature_array = feature_column
        self.dense_dim, self.sparse_dim = len(dense_feature_array), len(sparse_feature_array) # 特征维度
        self.tot_dims = len(dense_feature_array) # 计数的是dense特征 + embedding 维度
        # embed_map
        self.sparse_embed_map = {}
        for idx, feat in enumerate(sparse_feature_array):
            key = 'embed_' + str(idx)
            self.sparse_embed_map[key] = \
                keras.layers.Embedding(
                                         input_dim=feat['feat_num'],
                                         input_length=1, # 指名进行 embeding 的序列长度
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=keras.regularizers.l2(embed_reg)
                                    )
            self.tot_dims += feat['embed_dim']
        # 交叉网络 cross
        self.crossnet = Cross(cross_layers, self.tot_dims)
        # deep部分
        self.deepnet = DNN(hidden_units, activation=activation, dropout=dnn_dropout, **kwgs)
    
    
    def call(self, inputs):
        dense_input, sparse_input = inputs
        # 处理 sparse feature
        res = []
        for i in range(sparse_input.shape[1]):
            embed_layer = self.sparse_embed_map['embed_'+str(i)]
            res.append(embed_layer(sparse_input[:, i]))
        embed_feat = tf.concat(res, axis=-1) # 得到稠密的embeding向量
        feats = tf.concat([embed_feat, dense_input], axis=-1) # 得到cross net 的输入向量
        
        # cross
        cross_out = self.crossnet(feats)
        
        # deep
        deep_out = self.deepnet(feats)
        
        # 最终特征
        x = tf.concat([cross_out, deep_out], axis=-1)
        final_layer = keras.layers.Dense(1, activation='sigmoid') # 开启使用 bias
        return final_layer(x)
    
    
    def build_graph(self):
        dense_input = keras.Input(shape=(self.dense_dim,), dtype=tf.float32)
        sparse_input = keras.Input(shape=(self.sparse_dim,), dtype=tf.float32)
        model = keras.Model(inputs=[dense_input, sparse_input],
                           outputs=self.call([dense_input, sparse_input]))
        return model

In [636]:
keras.backend.clear_session()
sample_num = 100000
test_size = 0.2

embed_dim = 8
dnn_dropout = 0.5
hidden_units = [256, 128, 64]
cross_layers=3

learning_rate = 0.001
batch_size = 512
epochs = 5
    
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
# mirrored_strategy = tf.distribute.MirroredStrategy() # 分布式训练策略
# with mirrored_strategy.scope():
cdl = CrossDeep(feature_columns, cross_layers=cross_layers, hidden_units=hidden_units, dnn_dropout=dnn_dropout)
model = cdl.build_graph()
# ============================Compile============================
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=[keras.metrics.AUC()])


model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],  # checkpoint
    batch_size=batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])


Epoch 1/5
141/141 [==============================] - 33s 219ms/step - loss: 1.8291 - auc: 0.6336 - val_loss: 0.5700 - val_auc: 0.7308
Epoch 2/5
141/141 [==============================] - 32s 227ms/step - loss: 0.4842 - auc: 0.7767 - val_loss: 0.4956 - val_auc: 0.7423
Epoch 3/5
141/141 [==============================] - 31s 223ms/step - loss: 0.4024 - auc: 0.8640 - val_loss: 0.5480 - val_auc: 0.7236
Epoch 4/5
141/141 [==============================] - 31s 223ms/step - loss: 0.3039 - auc: 0.9335 - val_loss: 0.6283 - val_auc: 0.7101
Epoch 5/5
40/40 [==============================] - 4s 102ms/step - loss: 0.7222 - auc: 0.6854
test AUC: 0.685433


# XDeepFM

In [53]:
from tensorflow import keras 


class DNN(keras.layers.Layer):
    def __init__(self, hidden_units, activation='relu', dropout=0.1, reg_l2=0.01,**kwgs):
        super().__init__(**kwgs)
        self.dnn_network = [keras.layers.Dense(i, activation=activation, 
                                               kernel_regularizer=keras.regularizers.l2(reg_l2),  
                                               bias_regularizer=keras.regularizers.l2(reg_l2)) for i in hidden_units] # 使用正则化可以防止过拟合
        self.dropout = keras.layers.Dropout(dropout)
    
    def call(self, inputs):
        x = inputs
        for layer in self.dnn_network:
            x = layer(x)
        x = self.dropout(x)
        return x

class CIN(keras.layers.Layer):
    def __init__(self, layer_sizes, l2_reg=0.0001):
        """
        layer_nums : cross network的层数
        input_length: cross network的输入维度 
        """
        super().__init__()
        self.layer_sizes = layer_sizes
        self.l2_reg = l2_reg
        self.w = {}
    
    def build(self, input_shape):
        """设置参数共享变量"""
        x0 = input_shape[1]
        self.hidden_dims = [x0] + list(self.layer_sizes)
        for i in range(1, len(self.hidden_dims)):
            self.w['cin_w_'+str(i)] = self.add_weight(
                name='cin_w_'+str(i),
                shape=(1, self.hidden_dims[0]*self.hidden_dims[i-1], self.hidden_dims[i]), # shape [filter_height, filter_width, n_filters]
                initializer=keras.initializers.random_uniform,
                regularizer=keras.regularizers.l2(self.l2_reg),
                trainable=True
            )
        super().build(input_shape)
    
    
    def call(self, inputs):
        """
        输入的是稀疏的类别变量经过embedding的结果
        inputs： [batch, fileds, embedding]
        """
        embed_dims = inputs.shape[-1]
        x = inputs
        # 在 embedding 维度进行切割，好统计每一个embedding的子维度下，任意两个embedding的乘积情况
        split_x0 = tf.split(x, embed_dims, 2) # 放回的是一个列表，共embed_dims个元素，每个元素是张量为[batch, fileds0, 1]

        layers_out = [x] # last_output[i] 代表第 i 层CIN的输出
        for i in range(1, len(self.hidden_dims)):
            # 获取上一层的输出
            split_xk = tf.split(layers_out[-1], embed_dims, 2) # embed_dims个元素，每个张量为 [batch, h_i_1, 1]
            # tf.matmul 只作用在张量的最后两维上进行矩阵乘法
            out_ = tf.matmul(split_x0, split_xk, transpose_b=True) # [embed_dims, batch, fileds0, h_(i-1)]
            out_ = tf.reshape(out_, shape=(embed_dims, -1, out_.shape[-1]*out_.shape[-2])) # [embed_dims, batch, fileds0 * h_(i-1)]
            out_ = tf.transpose(out_, perm=[1, 0, 2]) # [batch, embed_dims, fileds0 * h_(i-1)]
            
#             print(out_.shape, self.w['cin_w_%d'%i].shape)
            # 由卷积公式得到第 i层的输出
            after_conv = tf.nn.conv1d(input=out_, filters=self.w['cin_w_%d'%i], stride=1, padding='VALID') # [batch, embed_dims, n_filters]
            #调整shape的顺序
            out_ = tf.transpose(after_conv, perm=[0, 2, 1]) # [batch, n_filters, embed_dims]
            layers_out.append(out_)
        
        
        # 模拟 rnn 进行序列池化
        out_ = tf.concat(layers_out[1:], axis=1) # [batch, h1+h2+..hk, embed_dims]
        out_ = tf.reduce_sum(out_, axis=-1, keepdims=False) # [batch, h1+h2+...+hk]
        return out_
            
        

class XDeepFM(keras.Model):
    def __init__(self, feature_column, hidden_units, cin_layer_size=(128, 56), activation='relu', dnn_dropout=0, embed_reg=1e-4, cin_l2_reg=1e-4, **kwgs):
        super().__init__(**kwgs)
        dense_feature_array, sparse_feature_array = feature_column
        self.dense_dim, self.sparse_dim = len(dense_feature_array), len(sparse_feature_array) # 特征维度
        self.tot_dims = len(dense_feature_array) # 计数的是dense特征 + embedding 维度
        # embed_map
        self.sparse_embed_map = {}
        for idx, feat in enumerate(sparse_feature_array):
            key = 'embed_' + str(idx)
            self.sparse_embed_map[key] = \
                keras.layers.Embedding(
                                         input_dim=feat['feat_num'],
                                         input_length=1, # 指名进行 embeding 的序列长度
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=keras.regularizers.l2(embed_reg)
                                    )
            self.tot_dims += feat['embed_dim']
        # Linear
        self.linear = keras.layers.Dense(1, name='linear')
       # CIN
        self.cin = CIN(cin_layer_size, cin_l2_reg)
        self.cin_linear = keras.layers.Dense(1, name='cin_linear')
        # deep部分
        self.deepnet = DNN(hidden_units, activation=activation, dropout=dnn_dropout, **kwgs)
        self.dnn_linear = keras.layers.Dense(1, name='dnn_linear')
    
    
    def call(self, inputs):
        dense_input, sparse_input = inputs
        # 处理 sparse feature
        res = []
        for i in range(sparse_input.shape[1]):
            embed_layer = self.sparse_embed_map['embed_'+str(i)]
            res.append(embed_layer(sparse_input[:, i]))
        embed_feat = tf.concat(res, axis=-1) # 得到稠密的embeding向量
        feats = tf.concat([embed_feat, dense_input], axis=-1) # 得到cross net 的输入向量
        
        # Linear
        # 目前为了简单起见我这里只考虑了 dense 特征
        linear_logit = self.linear(dense_input)

        # deep
        deep_out = self.deepnet(feats)
        dnn_logits = self.dnn_linear(deep_out)
        
        # CIN, 与 DNN 共享 embedding
        res = []
        for i in range(sparse_input.shape[1]):
            embed_layer = self.sparse_embed_map['embed_'+str(i)]
            res.append(embed_layer(sparse_input[:, i])) # [batch, embedding_dims]
        
        print(res[-1].shape)
#         assert res[-1].ndim == 2
        cin_input = tf.stack(res, axis=1)
        cin_out = self.cin(cin_input)
        cin_logits = self.cin_linear(cin_out)
        
        tot = keras.layers.Concatenate()([linear_logit, dnn_logits, cin_logits])
        finlear = keras.layers.Dense(1, use_bias=False)
        return tf.nn.sigmoid(finlear(tot))
        
    
    def build_graph(self):
        dense_input = keras.Input(shape=(self.dense_dim,), dtype=tf.float32)
        sparse_input = keras.Input(shape=(self.sparse_dim,), dtype=tf.float32)
        model = keras.Model(inputs=[dense_input, sparse_input],
                           outputs=self.call([dense_input, sparse_input]))
        return model

In [54]:
keras.backend.clear_session()
sample_num = 100000
test_size = 0.2

embed_dim = 8
dnn_dropout = 0.5
hidden_units = [256, 128, 64]
cross_layers=3

learning_rate = 0.001
batch_size = 512
epochs = 5
    
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
# mirrored_strategy = tf.distribute.MirroredStrategy() # 分布式训练策略
# with mirrored_strategy.scope():
xdfm = XDeepFM(feature_columns, cin_layer_size=(128, 56), hidden_units=hidden_units, dnn_dropout=dnn_dropout)
model = xdfm.build_graph()
# ============================Compile============================
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=[keras.metrics.AUC()])


model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],  # checkpoint
    batch_size=batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])


(None, 8)
Epoch 1/5
141/141 [==============================] - 22s 141ms/step - loss: 1.8504 - auc: 0.5995 - val_loss: 0.5791 - val_auc: 0.7222
Epoch 2/5
141/141 [==============================] - 20s 145ms/step - loss: 0.4899 - auc: 0.7807 - val_loss: 0.5072 - val_auc: 0.7350
Epoch 3/5
141/141 [==============================] - 21s 147ms/step - loss: 0.4046 - auc: 0.8724 - val_loss: 0.5562 - val_auc: 0.7182
Epoch 4/5
141/141 [==============================] - 21s 149ms/step - loss: 0.3096 - auc: 0.9354 - val_loss: 0.6410 - val_auc: 0.6971
Epoch 5/5
40/40 [==============================] - 2s 56ms/step - loss: 0.6700 - auc: 0.6973
test AUC: 0.697315


# DeepFM

In [735]:
class WideFM(keras.Model):
    """
    wide part
    实现的是一个应用在deepFM框架中的FM部分，与传统FM最大的不同是不需要分解得到特征隐向量
    计算二阶交互的时候直接将embedding向量作为原始特征隐向量计算内积即可
    """
    def __init__(self, feat_dims):
        # 一阶线性转换权重
        super().__init__()
        self.w = self.add_weight(shape=(feat_dims, 1),
                                initializer=keras.initializers.glorot_normal(),
                                trainable=True)
    
    def call(self, inputs):
        """
        inputs_1: dense_feature + embed_feat for calculate factor 1 #[batch, feat_dims] 
        inputs_2: embed_feat for calculate factor 2 [batch, fileds, embeding]
        """
        inputs1, inputs2 = inputs
        factor1 = tf.matmul(inputs1, self.w) # no bias [batch, 1]
        
        square_sum = tf.pow(tf.reduce_sum(inputs2, axis=1, keepdims=False), 2) # [batch,embedding]
        sum_square = tf.reduce_sum(tf.pow(inputs2, 2), axis=1, keepdims=False) # [batch, embedding]
        factor2 = 0.5*tf.reduce_sum(square_sum - sum_square, axis=1, keepdims=True) # [batch, 1]
#         print(inputs1.shape, inputs2.shape, factor1.shape, square_sum.shape, sum_square.shape, factor2.shape)
        return factor1 + factor2


class DNN(keras.layers.Layer):
    """
    deep 部分
    """
    def __init__(self, hidden_units, activation='relu', dropout=0.1, reg_l2=0.01,**kwgs):
        super().__init__(**kwgs)
        self.dnn_network = [keras.layers.Dense(i, activation=activation, 
                                               kernel_regularizer=keras.regularizers.l2(reg_l2),  
                                               bias_regularizer=keras.regularizers.l2(reg_l2)) for i in hidden_units] # 使用正则化可以防止过拟合
        self.dropout = keras.layers.Dropout(dropout)
    
    def call(self, inputs):
        x = inputs
        for layer in self.dnn_network:
            x = layer(x)
        x = self.dropout(x)
        return x



class DeepFM(keras.Model):
    def __init__(self, feature_column, hidden_units, activation='relu', dnn_dropout=0.1, reg_l2=0.01, embed_reg=0.01, **kwgs):
        super().__init__(**kwgs)
        dense_feature_array, sparse_feature_array = feature_column
        self.dense_dim, self.sparse_dim = len(dense_feature_array), len(sparse_feature_array) # 特征维度
        self.tot_dims = len(dense_feature_array) # 计数的是dense特征 + embedding 维度
        # embed_map
        self.sparse_embed_map = {}
        for idx, feat in enumerate(sparse_feature_array):
            key = 'embed_' + str(idx)
            self.embed_dims = feat['embed_dim'] # 稀疏特征的 embedding 维度
            self.sparse_embed_map[key] = \
                keras.layers.Embedding(
                                         input_dim=feat['feat_num'],
                                         input_length=1, # 指名进行 embeding 的序列长度
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=keras.regularizers.l2(embed_reg)
                                    )
            self.tot_dims += feat['embed_dim']
            
        # wide部分
        self.fm = WideFM(self.tot_dims)
        
        # deep部分
        self.dnn_network = DNN(hidden_units, activation, dnn_dropout, reg_l2)
        self.final_linear = keras.layers.Dense(1, activation=None) # 不使用偏执项
    
    
    def call(self, inputs):
        dense_input, sparse_input = inputs
        # 处理 sparse feature
        res = []
        for i in range(sparse_input.shape[1]):
            embed_layer = self.sparse_embed_map['embed_'+str(i)]
            res.append(embed_layer(sparse_input[:, i]))
        embed_feat = tf.concat(res, axis=-1) # 得到稠密的embeding向量
        feats = tf.concat([embed_feat, dense_input], axis=-1) # 得到稠密的输入向量
        second_input = tf.reshape(embed_feat, (-1, sparse_input.shape[1], self.embed_dims)) # 将类别的embeding tensor转换为三维 [batch, fileds, embed_dims]
        
#         print('first shape', feats.shape, 'second shape', second_input.shape)
        # wide
        out_fm = self.fm([feats, second_input])
#         print('out_fm shape', out_fm.shape)
        # deep部分
        out_deep = self.dnn_network(feats)
        out_deep = self.final_linear(out_deep)
#         print('out_deep shape', out_deep.shape)
        return tf.nn.sigmoid(out_deep + out_fm)
    
    
    def build_graph(self):
        dense_input = keras.Input(shape=(self.dense_dim,), dtype=tf.float32)
        sparse_input = keras.Input(shape=(self.sparse_dim,), dtype=tf.float32)
        model = tf.keras.Model(inputs=[dense_input, sparse_input],
                              outputs=self.call([dense_input, sparse_input]))
        return model
        

In [737]:
keras.backend.clear_session()
sample_num = 100000
test_size = 0.2

embed_dim = 8
dnn_dropout = 0.5
hidden_units = [256, 128, 64]
cross_layers=3

learning_rate = 0.001
batch_size = 512
epochs = 15
    
train_X, train_y = train
test_X, test_y = test
# ============================Build Model==========================
# mirrored_strategy = tf.distribute.MirroredStrategy() # 分布式训练策略
# with mirrored_strategy.scope():
dfm = DeepFM(feature_columns, hidden_units=hidden_units, dnn_dropout=dnn_dropout)
model = dfm.build_graph()
# ============================Compile============================
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=[keras.metrics.AUC()])


model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_auc', patience=5, restore_best_weights=True)],  # checkpoint
    batch_size=batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])


Epoch 1/15
141/141 [==============================] - 5s 21ms/step - loss: 3.4459 - auc: 0.5525 - val_loss: 0.6164 - val_auc: 0.6715
Epoch 2/15
141/141 [==============================] - 3s 19ms/step - loss: 0.5508 - auc: 0.6761 - val_loss: 0.5310 - val_auc: 0.6950
Epoch 3/15
141/141 [==============================] - 3s 19ms/step - loss: 0.5288 - auc: 0.6926 - val_loss: 0.5277 - val_auc: 0.7032
Epoch 4/15
141/141 [==============================] - 3s 20ms/step - loss: 0.5269 - auc: 0.6992 - val_loss: 0.5276 - val_auc: 0.7080
Epoch 5/15
141/141 [==============================] - 3s 22ms/step - loss: 0.5267 - auc: 0.7040 - val_loss: 0.5275 - val_auc: 0.7117
Epoch 6/15
141/141 [==============================] - 3s 21ms/step - loss: 0.5265 - auc: 0.7076 - val_loss: 0.5281 - val_auc: 0.7104
Epoch 7/15
141/141 [==============================] - 3s 22ms/step - loss: 0.5267 - auc: 0.7091 - val_loss: 0.5273 - val_auc: 0.7153
Epoch 8/15
141/141 [==============================] - 3s 22ms/step - 

# NFM

In [746]:
class DNN(keras.layers.Layer):
    """
    deep 部分
    """
    def __init__(self, hidden_units, activation='relu', dropout=0.1, reg_l2=0.01,**kwgs):
        super().__init__(**kwgs)
        self.dnn_network = [keras.layers.Dense(i, activation=activation, 
                                               kernel_regularizer=keras.regularizers.l2(reg_l2),  
                                               bias_regularizer=keras.regularizers.l2(reg_l2)) for i in hidden_units] # 使用正则化可以防止过拟合
        self.dropout = keras.layers.Dropout(dropout)
    
    def call(self, inputs):
        x = inputs
        for layer in self.dnn_network:
            x = layer(x)
        x = self.dropout(x)
        return x

    
class BiInteractionPoolling(keras.Model):
    def __init__(self):
        super().__init__()
    
    def call(self, x):
        # x shape = [batch, fileds, embed_dims]
        square_sum = tf.pow(tf.reduce_sum(x, axis=1), 2) # [batch, embed_dims]
        sum_square = tf.reduce_sum(tf.pow(x, 2), axis=1) #[batch, embed_dims]
        return 0.5*(square_sum - sum_square)

    
class NFM(keras.Model):
    def __init__(self, feature_column, hidden_units, activation='relu', dnn_dropout=0.1, reg_l2=0.01, embed_reg=0.01, **kwgs):
        super().__init__(**kwgs)
        dense_feature_array, sparse_feature_array = feature_column
        self.dense_dim, self.sparse_dim = len(dense_feature_array), len(sparse_feature_array) # 特征维度
        self.tot_dims = len(dense_feature_array) # 计数的是dense特征 + embedding 维度
        # embed_map
        self.sparse_embed_map = {}
        for idx, feat in enumerate(sparse_feature_array):
            key = 'embed_' + str(idx)
            self.embed_dims = feat['embed_dim'] # 稀疏特征的 embedding 维度
            self.sparse_embed_map[key] = \
                keras.layers.Embedding(
                                         input_dim=feat['feat_num'],
                                         input_length=1, # 指名进行 embeding 的序列长度
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=keras.regularizers.l2(embed_reg)
                                    )
            self.tot_dims += feat['embed_dim']
        
        # 低阶的特征整合
        self.first_linear = keras.layers.Dense(1, activation=None)
        
        # 高阶部分
        # 特征交互层
        self.bi = BiInteractionPoolling()
        # DNN
        self.dnn = DNN(hidden_units, activation, dnn_dropout, reg_l2)
        # BatchNormalization
        self.bn = keras.layers.BatchNormalization() # 作用在DNN网络的输入一端
        # dnn linear
        self.dnn_linear = keras.layers.Dense(1, activation=None)
    
    def call(self, inputs):
        dense_input, sparse_input = inputs
        res = []
        for i in range(sparse_input.shape[1]):
            key = 'embed_' + str(i)
            embed = self.sparse_embed_map[key]
            res.append(embed(sparse_input[:,i:i+1]))
        embed_feats = tf.concat(res, axis=1)  # [batch, fileds, embed_dims]
        
        # 一阶信息
        into_first = tf.reshape(embed_feats, (-1, embed_feats.shape[-1]*embed_feats.shape[1]))
        out_first = self.first_linear(into_first)
        
        # 二阶信息的提取
        # bi-interaction
        out_bi = self.bi(embed_feats) # [batch, embed_dims]
        # concat dense_feature
        into_dnn = tf.concat([out_bi, dense_input], axis=-1) #[batch, feats]
        # batchnormalize
        into_dnn = self.bn(into_dnn)
        out_dnn = self.dnn(into_dnn)
        out_dnn = self.dnn_linear(out_dnn) # [batch, 1]
        
        return tf.nn.sigmoid(out_first + out_dnn)
    
    
    def build_graph(self):
        dense_input = keras.Input(shape=(self.dense_dim,), dtype=tf.float32)
        sparse_input = keras.Input(shape=(self.sparse_dim,), dtype=tf.float32)
        model = keras.Model(inputs=[dense_input, sparse_input], outputs=self.call([dense_input, sparse_input]))
        return model
        

In [747]:
keras.backend.clear_session()
sample_num = 100000
test_size = 0.2

embed_dim = 8
dnn_dropout = 0.5
hidden_units = [256, 128, 64]
cross_layers=3

learning_rate = 0.001
batch_size = 512
epochs = 15
    
train_X, train_y = train
test_X, test_y = test

nfm = NFM(feature_columns, hidden_units=hidden_units, dnn_dropout=dnn_dropout)
model = nfm.build_graph()
# ============================Compile============================
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=[keras.metrics.AUC()])


model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_auc', patience=5, restore_best_weights=True)],  # checkpoint
    batch_size=batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])


Epoch 1/15
141/141 [==============================] - 5s 22ms/step - loss: 3.0375 - auc: 0.6382 - val_loss: 0.6702 - val_auc: 0.7002
Epoch 2/15
141/141 [==============================] - 3s 19ms/step - loss: 0.5715 - auc: 0.7205 - val_loss: 0.5787 - val_auc: 0.7269
Epoch 3/15
141/141 [==============================] - 3s 19ms/step - loss: 0.5482 - auc: 0.7310 - val_loss: 0.5665 - val_auc: 0.7320
Epoch 4/15
141/141 [==============================] - 3s 19ms/step - loss: 0.5432 - auc: 0.7391 - val_loss: 0.5455 - val_auc: 0.7376
Epoch 5/15
141/141 [==============================] - 3s 19ms/step - loss: 0.5390 - auc: 0.7475 - val_loss: 0.5435 - val_auc: 0.7412
Epoch 6/15
141/141 [==============================] - 3s 20ms/step - loss: 0.5377 - auc: 0.7534 - val_loss: 0.5460 - val_auc: 0.7347
Epoch 7/15
141/141 [==============================] - 3s 20ms/step - loss: 0.5346 - auc: 0.7619 - val_loss: 0.5443 - val_auc: 0.7398
Epoch 8/15
141/141 [==============================] - 3s 20ms/step - 

# AFM

In [755]:
import itertools


class DNN(keras.layers.Layer):
    """
    deep 部分
    """
    def __init__(self, hidden_units, activation='relu', dropout=0.1, reg_l2=0.01,**kwgs):
        super().__init__(**kwgs)
        self.dnn_network = [keras.layers.Dense(i, activation=activation, 
                                               kernel_regularizer=keras.regularizers.l2(reg_l2),  
                                               bias_regularizer=keras.regularizers.l2(reg_l2)) for i in hidden_units] # 使用正则化可以防止过拟合
        self.dropout = keras.layers.Dropout(dropout)
    
    def call(self, inputs):
        x = inputs
        for layer in self.dnn_network:
            x = layer(x)
        x = self.dropout(x)
        return x

class AttentionLayer(keras.Model):
    def __init__(self, k, embed_dims):
        """
        k 表示的是attention层的隐藏神经元个数
        """
        super().__init__()
        self.w = self.add_weight(shape=(embed_dims, k),
                                initializer=keras.initializers.glorot_normal(),
                                trainable=True)
        self.bias = self.add_weight(shape=(k, ),
                                initializer=keras.initializers.glorot_normal(),
                                trainable=True)
        self.h = self.add_weight(shape=(k, 1),
                                initializer=keras.initializers.glorot_normal(),
                                trainable=True)
    def call(self, x):
        """
        输入 x 的shape为 [batch, (fileds*(fileds-1)/2), embed_dims]
        """
        a = tf.matmul(x, self.w) + self.bias # [batch, (fileds*(fileds-1)/2), k]
        a = tf.nn.relu(a) # [batch, (fileds*(fileds-1)/2), k]
        a = tf.matmul(a, self.h) # # [batch, (fileds*(fileds-1)/2), 1] 
        
        # 添加权重
        x = x * a # 元素积 # [batch, (fileds*(fileds-1)/2), embed_dims]
        # pooling
        return tf.reduce_sum(x, axis=1) # [batch, embed_dims] 使用 add 池化


class AFM(keras.Model):
    def __init__(self, feature_column, hidden_units, attention_dim=20, activation='relu', dnn_dropout=0.1, reg_l2=0.01, embed_reg=0.01, **kwgs):
        super().__init__(**kwgs)
        dense_feature_array, sparse_feature_array = feature_column
        self.dense_dim, self.sparse_dim = len(dense_feature_array), len(sparse_feature_array) # 特征维度
        self.tot_dims = len(dense_feature_array) # 计数的是dense特征 + embedding 维度
        # embed_map
        self.sparse_embed_map = {}
        for idx, feat in enumerate(sparse_feature_array):
            key = 'embed_' + str(idx)
            self.embed_dims = feat['embed_dim'] # 稀疏特征的 embedding 维度
            self.sparse_embed_map[key] = \
                keras.layers.Embedding(
                                         input_dim=feat['feat_num'],
                                         input_length=1, # 指名进行 embeding 的序列长度
                                         output_dim=feat['embed_dim'],
                                         embeddings_initializer='random_uniform',
                                         embeddings_regularizer=keras.regularizers.l2(embed_reg)
                                    )
            self.tot_dims += feat['embed_dim']
        
        # 低阶的特征整合
        self.first_linear = keras.layers.Dense(1, activation=None)
        
        # attention layer
        self.att = AttentionLayer(attention_dim, self.embed_dims)
        
        # dnn 
        self.dnn = DNN(hidden_units, activation, dnn_dropout, reg_l2)
        # bn 层
        self.bn = tf.keras.layers.BatchNormalization()
        # dnn_linear
        self.dnn_linear = keras.layers.Dense(1, activation=None)
    
    
    def call(self, inputs):
        dense_input, sparse_input = inputs
        res = []
        for i in range(sparse_input.shape[1]):
            key = 'embed_' + str(i)
            embed = self.sparse_embed_map[key]
            res.append(embed(sparse_input[:,i:i+1]))
        embed_feats = tf.concat(res, axis=1)  # [batch, fileds, embed_dims]
        
        # 一阶信息
        into_first = tf.reshape(embed_feats, (-1, embed_feats.shape[-1]*embed_feats.shape[1]))
        out_first = self.first_linear(into_first)
        
        # 二阶需要先计算两两embedding元素积的结果
        left = [] # 记录的是元素积的左部分embedding向量
        right = []
        for i, j in itertools.combinations(range(sparse_input.shape[1]), 2): # 得到元素积索引组合的情况
            left.append(i)
            right.append(j)
        # 采用切片的手法，来得到新的索引
        l = tf.gather(embed_feats, left, axis=1) # [batch, (fileds*(fileds-1)/2), embed_dims]
        r = tf.gather(embed_feats, right, axis=1) # [batch, (fileds*(fileds-1)/2), embed_dims]
        # 使用张量计算直接得到fileds*(fileds-1)/2个元素积
        into_second = l * r # [batch, (fileds*(fileds-1)/2), embed_dims]
        into_dnn = self.att(into_second) # [batch, embed_dims]
        # 与连续特征做拼接
        into_dnn = tf.concat([into_dnn, dense_input], axis=-1)
        # 进入 bn 层防止过拟合
        into_dnn = self.bn(into_dnn)
        x = self.dnn(into_dnn)
        out_second = self.dnn_linear(x)
        
        # 苹姐一阶和高阶的输出
        return tf.nn.sigmoid(out_first + out_second)

    
    def build_graph(self):
        dense_input = keras.Input(shape=(self.dense_dim,), dtype=tf.float32)
        sparse_input = keras.Input(shape=(self.sparse_dim,), dtype=tf.float32)
        model = keras.Model(inputs=[dense_input, sparse_input], outputs=self.call([dense_input, sparse_input]))
        return model

In [756]:
keras.backend.clear_session()
sample_num = 100000
test_size = 0.2

embed_dim = 8
dnn_dropout = 0.5
hidden_units = [256, 128, 64]
cross_layers=3

learning_rate = 0.001
batch_size = 512
epochs = 15
    
train_X, train_y = train
test_X, test_y = test

afm = AFM(feature_columns, hidden_units=hidden_units, dnn_dropout=dnn_dropout)
model = afm.build_graph()
# ============================Compile============================
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=[keras.metrics.AUC()])


model.fit(
    train_X,
    train_y,
    epochs=epochs,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_auc', patience=5, restore_best_weights=True)],  # checkpoint
    batch_size=batch_size,
    validation_split=0.1
)
# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])


Epoch 1/15
141/141 [==============================] - 12s 63ms/step - loss: 3.0184 - auc: 0.5809 - val_loss: 0.6360 - val_auc: 0.6819
Epoch 2/15
141/141 [==============================] - 9s 60ms/step - loss: 0.5603 - auc: 0.7129 - val_loss: 0.5785 - val_auc: 0.7247
Epoch 3/15
141/141 [==============================] - 9s 62ms/step - loss: 0.5410 - auc: 0.7391 - val_loss: 0.5625 - val_auc: 0.7341
Epoch 4/15
141/141 [==============================] - 9s 64ms/step - loss: 0.5391 - auc: 0.7526 - val_loss: 0.5486 - val_auc: 0.7382
Epoch 5/15
141/141 [==============================] - 9s 67ms/step - loss: 0.5400 - auc: 0.7617 - val_loss: 0.5443 - val_auc: 0.7416
Epoch 6/15
141/141 [==============================] - 9s 66ms/step - loss: 0.5383 - auc: 0.7720 - val_loss: 0.5505 - val_auc: 0.7402
Epoch 7/15
141/141 [==============================] - 9s 66ms/step - loss: 0.5390 - auc: 0.7774 - val_loss: 0.5528 - val_auc: 0.7390
Epoch 8/15
141/141 [==============================] - 9s 67ms/step -

# DIN

### DIN初探，基于伪造的数据集

In [927]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow import keras as keras
from keras import layers

# 构建DIN模型

# Dice自适应激活函数
class Dice(layers.Layer):
    def __init__(self):
        super().__init__()
        self.alpha = self.add_weight(shape=(), initializer=keras.initializers.Zeros(), dtype=tf.float32)
        self.bn = layers.BatchNormalization(center=False, scale=False, trainable=True) # 只使用BN的归一化，不学习移动和缩放参数

    def call(self, x):
        p = tf.nn.sigmoid(self.bn(x))
        return p*x + (1-p)*self.alpha*x


class ActivationLayer(layers.Layer):
    """
    注意力层，用于加权融合用户历史行为序列
    """
    def __init__(self, att_hidden_units, activation='prelu'):
        super().__init__()
        self.att_dense = [layers.Dense(unit, activation=Dice() if ffn_activation != 'prelu' else layers.PReLU()) for unit in att_hidden_units]
        self.att_final_dense = layers.Dense(1)

    def call(self, inputs):

        """
        inputs [query_items, hist_items, mask]
        quer_items : [batch, items_embedding_dims]
        hist_items: [batch, seq_len, items_embedding_dims]
        mask : [batch, seq_len] 因为用户序列是不定长的所以使用mask掩掉那些填充的序列维度
        """
        q, k, mask = inputs
        val = tf.identity(k) # copy
        q = tf.tile(q, multiples=[1, k.shape[1]])  # [batch, seq_len*items_embedding_dims]
        q = tf.reshape(q, shape=(-1, k.shape[1], k.shape[2]))  # [batch, seq_len, items_embedding_dims]

        x = tf.concat([q, k, q-k, q*k], axis=-1)  # [batch, seq_len, 4*items_embedding_dims]
        for layer in self.att_dense:
            x = layer(x)
        # x shape 为 [batch, seq_len, last_hidden_units]
        w = self.att_final_dense(x)  # [batch, seq_len, 1]
        w = tf.squeeze(w, axis=-1)  # [batch, seq_len]

        w = tf.where(tf.equal(mask, 0), -(1<<31)*1.0, w)  # [batch, seq_len] 将mask位置的权重调整为很大的负数，在经过softmax之后权重会变为0

        # 使用 softmax 归一化权重
        w = tf.nn.softmax(w, axis=-1)  # [batch, seq_len]
        w = tf.expand_dims(w, axis=1)  # [batch, 1, seq_len]
        # 注意力的体现加权融合池化用户历史序列特征
        outputs = tf.matmul(w, val)  # [batch, 1, items_embedding_dims]
        return tf.squeeze(outputs, axis=1)  # [batch, items_embedding_dims]


class DIN(keras.layers.Layer):
    def __init__(self, sparse_feature_dict, sparse_feature_index, att_hidden_units=(80, 40),
                 ffn_hidden_units=(80, 40), att_activation='prelu', ffn_activation='prelu', maxlen=10, dnn_dropout=0.,
                 embed_reg=1e-4):
        super().__init__()
        self.maxlen = maxlen
        # self.sparse_feature_dict = sparse_feature_dict # 存取了所有类别特征需要 embedding 的相关信息
        self.user_sparse_feature_index, self.item_sparse_feature_index, self.behavior_feature_index = sparse_feature_index
        self.embed_layers = {
            'embed_' + k: layers.Embedding(v[0], v[1], embeddings_regularizer=keras.regularizers.l2(embed_reg))
            for k, v in sparse_feature_dict.items()
        }

        self.attention = ActivationLayer(att_hidden_units, activation=Dice() if att_activation != 'prelu' else layers.PReLU())
        self.bn = layers.BatchNormalization()  # BN
        # MLP 结构提取高教特征交互信息
        self.ffn = keras.Sequential([layers.Dense(i, activation=Dice() if ffn_activation != 'prelu' else layers.PReLU())
                                     for i in ffn_hidden_units])
        self.dropout = layers.Dropout(dnn_dropout)
        self.final_output = layers.Dense(1)


    def call(self, inputs):
        # 需要事先对behavior中不等长的序列进行填充为等长，
        # inputs的输入是不考虑batch维度的
        # mask [seq_len]
        dense_user_feat, sparse_user_feat, dense_item_feat, sparse_item_feat, behavior_feat, mask = inputs
        embed_user_feat = tf.concat([
            self.embed_layers['embed_'+k](sparse_user_feat[:, i])
            for k, i in self.user_sparse_feature_index.items()
        ], axis=-1)  # [batch, user_embed_dims]
        user_embed = tf.concat([dense_user_feat, embed_user_feat], axis=-1)  # [batch, user_tot_dims]

        embed_item_feat = tf.concat([
            self.embed_layers['embed_'+k](sparse_item_feat[:, i])
            for k, i in self.item_sparse_feature_index.items()
        ], axis=-1)  # [batch, item_embed_dims]
        item_embed = tf.concat([dense_item_feat, embed_item_feat], axis=-1)  # [batch, item_tot_dims]

        embed_behaviors = tf.concat([
            # 使用物品的embedding向量来得到用户的历史embedding序列
            self.embed_layers['embed_{v[0]}_{v[1]}_{v[3]}'.format(v=key.split('_'))](behavior_feat[:, i])
            for key, i in self.behavior_feature_index.items()
        ], axis=-1)  # [batch, tot_seq_embed_dims]

        embed_behaviors = tf.reshape(embed_behaviors, shape=(-1, self.maxlen, embed_item_feat.shape[1]))  # [batch, seq_len, item_embed_dims]
        mask = tf.reshape(mask, shape=[-1, self.maxlen]) # [batch, seq_len]
        behavior_out = self.attention([embed_item_feat, embed_behaviors, mask]) # [batch, items_embedding_dims]
        # print('behavior,', behavior_out.shape)
        tot_feat = tf.concat([user_embed, item_embed, behavior_out], axis=-1) # [batch, feat_dims]
        tot_feat = self.bn(tot_feat)
        out = self.ffn(tot_feat)
        out = self.dropout(out)
        out = self.final_output(out)
        print(out.shape)
        return tf.nn.sigmoid(out) # [batch, 1]

    def build_graph(self):
        dense_user_input = keras.Input(shape=(5,), dtype=tf.float32)
        sparse_user_input = keras.Input(shape=(3,), dtype=tf.float32)
        dense_item_input = keras.Input(shape=(5,), dtype=tf.float32)
        sparse_item_input = keras.Input(shape=(3,), dtype=tf.float32)
        behavior_input = keras.Input(shape=(3*self.maxlen,), dtype=tf.float32)
        mask = keras.Input(shape=(self.maxlen,), dtype=tf.float32)

        model = keras.Model(inputs=[dense_user_input, sparse_user_input, dense_item_input, sparse_item_input, behavior_input, mask],
                            outputs=self.call([dense_user_input, sparse_user_input, dense_item_input, sparse_item_input, behavior_input, mask]))

        return model




In [928]:
# 伪造随机数据进行测试，
if __name__ == '__main__':
    maxlen = 10

    embed_dim = 8
    att_hidden_units = [80, 40]
    ffn_hidden_units = [256, 128, 64]
    dnn_dropout = 0.5
    att_activation = 'sigmoid'
    ffn_activation = 'prelu'

    learning_rate = 0.001
    batch_size = 64
    epochs = 5

    user_dense_feature_train = pd.DataFrame(np.random.random((10000, 5)),
                                            columns=['user_dense_{}'.format(i) for i in range(5)])
    user_sparse_feature_train = pd.DataFrame(np.random.randint(1, 10, size=(10000, 3)),
                                             columns=['user_sparse_{}'.format(i) for i in range(3)])
    item_dense_feature_train = pd.DataFrame(np.random.random((10000, 5)),
                                            columns=['item_dense_{}'.format(i) for i in range(5)])
    item_sparse_feature_train = pd.DataFrame(np.random.randint(1, 10, size=(10000, 3)),
                                             columns=['item_sparse_{}'.format(i) for i in range(3)])
    behavior_feature_train = None


    for ml in range(maxlen): # 序列长度
        tmp = pd.DataFrame(np.random.randint(1, 10, size=(10000, 3)),
                           columns=['item_sparse_{}_{}'.format(ml, i) for i in range(3)])
        if ml == 0:
            behavior_feature_train = tmp
        else:
            behavior_feature_train = pd.concat([behavior_feature_train, tmp], axis=1)

    # 模拟每个用户的序列是不定长的
    mask_train = []
    for i in range(10000):
        tmp = [1]*maxlen
        idx = np.random.randint(0, maxlen+1)
        tmp[idx:] = [0]*(maxlen-idx) # 用0表示是掩码位置，Attention中应该将权重置为1
        mask_train.append(tmp)

    mask_train = tf.constant(mask_train, dtype=tf.int8)
    print(mask_train[:5])

    # 模拟序列
    target_train = pd.DataFrame(np.random.randint(0, 2, size=10000))

    # valid
    user_dense_feature_val = pd.DataFrame(np.random.random((10000, 5)),
                                          columns=['user_dense_{}'.format(i) for i in range(5)])
    user_sparse_feature_val = pd.DataFrame(np.random.randint(1, 10, size=(10000, 3)),
                                           columns=['user_sparse_{}'.format(i) for i in range(3)])
    item_dense_feature_val = pd.DataFrame(np.random.random((10000, 5)),
                                          columns=['item_dense_{}'.format(i) for i in range(5)])
    item_sparse_feature_val = pd.DataFrame(np.random.randint(1, 10, size=(10000, 3)),
                                           columns=['item_sparse_{}'.format(i) for i in range(3)])
    behavior_feature_val = None
    for ml in range(maxlen):  # 行为序列的长度
        tmp = pd.DataFrame(np.random.randint(1, 10, size=(10000, 3)),  # 3 表征的是在序列的一个时间点上，有3个维度的特征
                           columns=['item_sparse_{}_{}'.format(ml, i) for i in range(3)])  # ml_i 表示的是序列ml中的第 i 个类别特征
        if ml == 0:
            behavior_feature_val = tmp
        else:
            behavior_feature_val = pd.concat([behavior_feature_val, tmp], axis=1)

    print(behavior_feature_val.shape)

    # 模拟每个用户的序列是不定长的
    mask_test = []
    for i in range(10000):
        tmp = [1] * maxlen
        idx = np.random.randint(0, maxlen + 1)
        tmp[idx:] = [0] * (maxlen - idx)  # 用0表示是掩码位置，Attention中应该将权重置为1
        mask_test.append(tmp)

    mask_test = tf.constant(mask_test, dtype=tf.int8)
    print(mask_test[:5])

    target_val = pd.DataFrame(np.random.randint(0, 2, size=10000))

    sparse_feature_dict = {}
    user_sparse_feature_index = {}
    item_sparse_feature_index = {}
    behavior_feature_index = {}

    for idx, col in enumerate(user_sparse_feature_train.columns):
        sparse_feature_dict[col] = (user_sparse_feature_train[col].max() + 1, embed_dim)  # embedding(特征种类数， embedding_dim)
        user_sparse_feature_index[col] = idx
    for idx, col in enumerate(item_sparse_feature_train.columns):
        sparse_feature_dict[col] = (item_sparse_feature_train[col].max() + 1, embed_dim)
        item_sparse_feature_index[col] = idx
    for idx, col in enumerate(behavior_feature_train.columns):
        behavior_feature_index[col] = idx

    sparse_feature_index = [user_sparse_feature_index, item_sparse_feature_index, behavior_feature_index]

    din = DIN(sparse_feature_dict, sparse_feature_index, att_hidden_units, ffn_hidden_units, att_activation,
              ffn_activation, maxlen, dnn_dropout)
    model = din.build_graph()
    # model.summary()
    # ============================model checkpoint======================
    check_path = 'save/din_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
                                                    verbose=1, period=5)
    # =========================Compile============================
    model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=[keras.metrics.AUC()])
    model.run_eagerly = True
    model.fit([user_dense_feature_train, user_sparse_feature_train, item_dense_feature_train, item_sparse_feature_train,
               behavior_feature_train, mask_train],
              target_train,
              epochs=epochs,
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True),
                         checkpoint],  # checkpoint
              validation_data=([user_dense_feature_val, user_sparse_feature_val, item_dense_feature_val,
                                item_sparse_feature_val, behavior_feature_val, mask_test], target_val),
              batch_size=batch_size,
              )


tf.Tensor(
[[1 1 1 1 1 1 1 1 1 0]
 [1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0]
 [1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0]], shape=(5, 10), dtype=int8)
(10000, 30)
tf.Tensor(
[[1 1 1 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 0 0]
 [1 1 1 1 1 0 0 0 0 0]], shape=(5, 10), dtype=int8)
(None, 1)
Epoch 1/5
157/157 [==============================] - 17s 107ms/step - loss: 0.7002 - auc_1: 0.5047 - val_loss: 0.6935 - val_auc_1: 0.4921
Epoch 2/5
157/157 [==============================] - 17s 106ms/step - loss: 0.6908 - auc_1: 0.5421 - val_loss: 0.6936 - val_auc_1: 0.5045


### 真实数据建模---亚马逊电商数据集

In [911]:
thr = 500000 # 采样的数据数目

# 数据处理部分
def to_df(file_path, thr=float('inf')):
    """
    转化为DataFrame结构
    :param file_path: 文件路径
    :return:
    """
    with open(file_path, 'r') as fin:
        df = {}
        i = 0
        for line in fin:
            df[i] = eval(line)
            i += 1
            if i >= thr: break #采样1w条
        df = pd.DataFrame.from_dict(df, orient='index')
        return df

def build_map(df, col_name):
    """
    制作一个映射，键为列名，值为序列数字
    :param df: reviews_df / meta_df
    :param col_name: 列名
    :return: 字典，键
    """
    key = sorted(df[col_name].unique().tolist())
    m = dict(zip(key, range(1, 1+len(key)))) # 从1开始进行label_encode, 0作为特殊的padding scalar
    df[col_name] = df[col_name].map(lambda x: m[x])
    return m, key

print('==========Data Preprocess Start============')
reviews_df = to_df('../data/Electronics_10.json', thr) # 只需要对 review 做采样， meta的数量级较小
meta_df = to_df('../data/meta_Electronics.json')


==========Data Preprocess Start============


In [912]:
reviews_df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IDCSC6NVONIZ,0972683275,2Cents!,"[1, 1]",This mount is just what I needed. It is stron...,5.0,Perfect,1367280000,"04 30, 2013"
1,A3BMUBUC1N77U8,0972683275,ahoffoss,"[0, 0]",This mount works really well once you get it u...,4.0,"Pretty simple, but definitely good!",1385164800,"11 23, 2013"
2,AQBLWW13U66XD,0972683275,"Benjamin Belanger ""v dbl u""","[0, 0]","I bought this for a 22"" TV for my son. I mount...",5.0,High Quality/Low Price,1375574400,"08 4, 2013"
3,A3IIGCFLKVFW8M,0972683275,"Brian M. Kaplan ""Brian M. Kaplan""","[0, 0]",Works great and is so much cheaper than the mo...,5.0,Holds a lot,1393459200,"02 27, 2014"
4,A3VKO21KYDJQ2W,0972683275,C. Aaland,"[0, 0]","For the price, you can't beat it. Mine didn't ...",3.0,Great for the price.,1310428800,"07 12, 2011"
...,...,...,...,...,...,...,...,...,...
347388,A3S3R88HA0HZG3,B00L3YHF6O,PT Cruiser,"[0, 0]",Bluetooth speakers have improved a lot over th...,5.0,"Oooh, aahh, ROAR!",1405468800,"07 16, 2014"
347389,A26VF18X91983P,B00L3YHF6O,Richard,"[0, 0]",Super sonic speaker system! Can you say that 5...,5.0,Simply stellar!,1405987200,"07 22, 2014"
347390,A2XRMQA6PJ5ZJ8,B00L3YHF6O,Roger J. Buffington,"[0, 0]",Disclosure: I received a free sample of this i...,5.0,Excellent Bluetooth speaker with lots of bells...,1404950400,"07 10, 2014"
347391,A3A4ZAIBQWKOZS,B00L3YHF6O,Stephen M. Lerch,"[18, 23]",My short review:If you have the money to spend...,5.0,Best sounding speaker at this price range,1404691200,"07 7, 2014"


In [913]:
meta_df

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
...,...,...,...,...,...,...,...,...,...
498191,BT008V9J9U,http://ecx.images-amazon.com/images/I/313e6SJm...,Vehicle suction cup mount (replacement) NOTICE...,"[[Electronics, GPS & Navigation, GPS System Ac...",Suction Cup Mount,21.99,NaN,{'buy_after_viewing': ['B000EPFCC2']},Garmin
498192,BT008SXQ4C,http://ecx.images-amazon.com/images/I/31oF9oNv...,Quatech - 1 Port PCMCIA to DB-25 Parallel Adap...,"[[Electronics, Computers & Accessories, Cables...",Parallel PCMCIA Card 1PORT Epp,23.99,NaN,"{'also_bought': ['B000SR2H4W', 'B001Q7X0W6'], ...",NaN
498193,BT008G3W52,http://ecx.images-amazon.com/images/I/21WIrX5f...,C2G - 5m Ultma USB 2.0 A Mini B Cble,"[[Electronics, Computers & Accessories, Cables...",C2G / Cables to Go 5M Ultima USB 2.0 Cable,18.91,NaN,"{'bought_together': ['B0002D6QJO'], 'buy_after...",C2G
498194,BT008UKTMW,http://ecx.images-amazon.com/images/I/41TNAVmf...,Keyboard drawer.,"[[Electronics, Computers & Accessories, Cables...",Underdesk Keyboard Drawer,25.54,NaN,"{'also_viewed': ['B0002LD0ZY', 'B0002LCZP0', '...",Fellowes


In [914]:
# 只保留reviews文件中出现过的商品
# 'reviewerID', 'asin', 'unixReviewTime' 分别表示用户id，物品id，购物的时间
meta_df = meta_df[meta_df['asin'].isin(reviews_df['asin'].unique())]
meta_df = meta_df.reset_index(drop=True)
reviews_df = reviews_df[['reviewerID', 'asin', 'unixReviewTime']]
meta_df = meta_df[['asin', 'categories']]
meta_df.shape, reviews_df.shape

((11589, 2), (347393, 3))

In [915]:
# 物品类别只保留最后一个
meta_df['categories'] = meta_df['categories'].map(lambda x: x[-1][-1])

In [916]:
# 高维类别数据的编号索引map
# meta_df文件的物品ID映射
asin_map, asin_key = build_map(meta_df, 'asin')
# meta_df文件物品种类映射
cate_map, cate_key = build_map(meta_df, 'categories') # 对meta_df 的 categories列 进行 label_encoder，并返回 map_dict 以及 encode_list 
# reviews_df文件的用户ID映射
revi_map, revi_key = build_map(reviews_df, 'reviewerID')

user_count, item_count, cate_count, example_count = \
    len(revi_map), len(asin_map), len(cate_map), reviews_df.shape[0]


<ipython-input-911-26aeec91a599>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = df[col_name].map(lambda x: m[x])


In [917]:
meta_df

,asin,categories
0,1,513
1,3,573
2,5,572
3,4,572
4,6,572
...,...,...
11584,11585,340
11585,11587,266
11586,11588,47
11587,11586,346


In [918]:
# reviews_df文件物品id进行映射，并按照用户id、浏览时间进行排序，重置索引
reviews_df['asin'] = reviews_df['asin'].map(lambda x: asin_map[x])
reviews_df = reviews_df.sort_values(['reviewerID', 'unixReviewTime'])
reviews_df = reviews_df.reset_index(drop=True)

<ipython-input-918-3d35b115f969>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['asin'] = reviews_df['asin'].map(lambda x: asin_map[x])


In [919]:
# 各个物品对应的类别
cate_list = np.array([0]+meta_df['categories'].to_list(), dtype='int32')

reviews_df.columns = ['user_id', 'item_id', 'time']

In [920]:
reviews_df

,user_id,item_id,time
0,1,885,1359936000
1,1,3892,1359936000
2,1,4477,1359936000
3,1,4709,1359936000
4,1,6541,1359936000
...,...,...,...
347388,20247,4913,1388534400
347389,20247,5052,1388534400
347390,20247,9047,1389744000
347391,20247,10490,1389744000


In [921]:
from tqdm import tqdm
import random

def gen_neg(pos_list):
    neg = pos_list[0]
    while neg in pos_list:
        neg = random.randint(0, item_count - 1) # 产生还没有点过的物品作为负例 [l, r]
    return neg

train_data, val_data, test_data = [], [], []

for user_id, hist in tqdm(reviews_df.groupby('user_id')):
    pos_list = hist['item_id'].tolist()

    neg_list = [gen_neg(pos_list) for i in range(len(pos_list))]
    hist = []
    for i in range(1, len(pos_list)):
        hist.append([pos_list[i - 1], cate_list[pos_list[i-1]]])
        hist_i = hist.copy()
        if i == len(pos_list) - 1:
            test_data.append([hist_i, [pos_list[i], cate_list[pos_list[i]]], 1]) # 历史行为，【正样本id,类别】，label
            test_data.append([hist_i, [neg_list[i], cate_list[neg_list[i]]], 0])# 历史行为，【负样本id,类别】，label
            # test_data.append([hist_i, [pos_list[i]], 1])
            # test_data.append([hist_i, [neg_list[i]], 0])
        elif i == len(pos_list) - 2:
            val_data.append([hist_i, [pos_list[i], cate_list[pos_list[i]]], 1])
            val_data.append([hist_i, [neg_list[i], cate_list[neg_list[i]]], 0])
            # val_data.append([hist_i, [pos_list[i]], 1])
            # val_data.append([hist_i, [neg_list[i]], 0])
        else:
            train_data.append([hist_i, [pos_list[i], cate_list[pos_list[i]]], 1])
            train_data.append([hist_i, [neg_list[i], cate_list[neg_list[i]]], 0])
            # train_data.append([hist_i, [pos_list[i]], 1])
            # train_data.append([hist_i, [neg_list[i]], 0])


# shuffle
random.shuffle(train_data)
random.shuffle(val_data)
random.shuffle(test_data)

100%|██████████████████████████████████████████████████████████████████████████| 20247/20247 [00:07<00:00, 2770.34it/s]


In [922]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen = 10
embed_dims = 8


# create dataframe
train = pd.DataFrame(train_data, columns=['hist', 'target_item', 'label'])
val = pd.DataFrame(val_data, columns=['hist', 'target_item', 'label'])
test = pd.DataFrame(test_data, columns=['hist', 'target_item', 'label'])

# if no dense or sparse features, can fill with 0
print('==================Padding===================')

sparse_dict = { # 配置 embedding层的信息
    'item_sparse_id': (item_count+1, embed_dims),
    'item_sparse_category': (cate_count+1, embed_dims)
}

user_sparse_index = {}

item_sparse_index = {
    'item_sparse_id': 0,
    'item_sparse_category': 1
}

behavior_index = {
}
for time in range(maxlen):
    for i, k in enumerate(['id', 'category']):
        key = f'item_sparse_{time}_{k}'
        behavior_index[key] = time*2 + i

# 如果没有dense 或者 稀疏 特征使用全 0 的tesor来代替
train_x = [np.zeros(shape=(len(train), )), np.zeros(shape=(len(train), )),
        np.zeros(shape=(len(train), )), np.array(list(train['target_item'].values)),
        tf.reshape(pad_sequences(train['hist'], maxlen=maxlen, value=0), shape=[len(train), -1]) # 用-1表示是填充的项
       ] # dense_user, sparse_user, dense_item, sparse_item, behavior_

val_x = [np.zeros(shape=(len(val), )), np.zeros(shape=(len(val), )),
        np.zeros(shape=(len(val), )), np.array(list(val['target_item'].values)),
        tf.reshape(pad_sequences(val['hist'], maxlen=maxlen, value=0), shape=[len(val), -1]) # 用-1表示是填充的项
       ] # dense_user, sparse_user, dense_item, sparse_item, behavior_

test_x = [np.zeros(shape=(len(test), )), np.zeros(shape=(len(test), )),
        np.zeros(shape=(len(test), )), np.array(list(test['target_item'].values)),
        tf.reshape(pad_sequences(test['hist'], maxlen=maxlen, value=0), shape=[len(test), -1]) # 用-1表示是填充的项
       ] # dense_user, sparse_user, dense_item, sparse_item, behavior_
print('============Data Preprocess End=============')


==================Padding===================
============Data Preprocess End=============


In [923]:
test_x[-1].shape

TensorShape([40494, 20])

### 搭建模型---为了适应本数据集进行了微调整

In [924]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow import keras as keras
from keras import layers

# Dice自适应激活函数
class Dice(layers.Layer):
    def __init__(self):
        super().__init__()
        self.alpha = self.add_weight(shape=(), initializer=keras.initializers.Zeros(), dtype=tf.float32)
        self.bn = layers.BatchNormalization(center=False, scale=False, trainable=True) # 只使用BN的归一化，不学习移动和缩放参数

    def call(self, x):
        p = tf.nn.sigmoid(self.bn(x))
        return p*x + (1-p)*self.alpha*x


class ActivationLayer(layers.Layer):
    """
    注意力层，用于加权融合用户历史行为序列
    """
    def __init__(self, att_hidden_units, activation='prelu'):
        super().__init__()
        self.att_dense = [layers.Dense(unit, activation=Dice() if ffn_activation != 'prelu' else layers.PReLU()) for unit in att_hidden_units]
        self.att_final_dense = layers.Dense(1)

    def call(self, inputs):

        """
        inputs [query_items, hist_items, mask]
        quer_items : [batch, items_embedding_dims]
        hist_items: [batch, seq_len, items_embedding_dims]
        mask : [batch, seq_len] 因为用户序列是不定长的所以使用mask掩掉那些填充的序列维度
        """
        q, k, mask = inputs
        val = tf.identity(k) # copy
        q = tf.tile(q, multiples=[1, k.shape[1]])  # [batch, seq_len*items_embedding_dims]
        q = tf.reshape(q, shape=(-1, k.shape[1], k.shape[2]))  # [batch, seq_len, items_embedding_dims]

        x = tf.concat([q, k, q-k, q*k], axis=-1)  # [batch, seq_len, 4*items_embedding_dims]
        for layer in self.att_dense:
            x = layer(x)
        # x shape 为 [batch, seq_len, last_hidden_units]
        w = self.att_final_dense(x)  # [batch, seq_len, 1]
        w = tf.squeeze(w, axis=-1)  # [batch, seq_len]

        w = tf.where(tf.equal(mask, 0), -(1<<31)*1.0, w)  # [batch, seq_len] 将mask位置的权重调整为很大的负数，在经过softmax之后权重会变为0

        # 使用 softmax 归一化权重
        w = tf.nn.softmax(w, axis=-1)  # [batch, seq_len]
        w = tf.expand_dims(w, axis=1)  # [batch, 1, seq_len]
        # 注意力的体现加权融合池化用户历史序列特征
        outputs = tf.matmul(w, val)  # [batch, 1, items_embedding_dims]
        return tf.squeeze(outputs, axis=1)  # [batch, items_embedding_dims]


class DIN(keras.layers.Layer):
    def __init__(self, sparse_feature_dict, sparse_feature_index, att_hidden_units=(80, 40),
                 ffn_hidden_units=(80, 40), att_activation='prelu', ffn_activation='prelu', maxlen=10, dnn_dropout=0.,
                 embed_reg=1e-4):
        super().__init__()
        self.maxlen = maxlen
        # self.sparse_feature_dict = sparse_feature_dict # 存取了所有类别特征需要 embedding 的相关信息
        self.user_sparse_feature_index, self.item_sparse_feature_index, self.behavior_feature_index = sparse_feature_index
        self.embed_layers = {
            'embed_' + k: layers.Embedding(v[0], v[1], embeddings_regularizer=keras.regularizers.l2(embed_reg))
            for k, v in sparse_feature_dict.items()
        }

        self.attention = ActivationLayer(att_hidden_units, activation=Dice() if att_activation != 'prelu' else layers.PReLU())
        self.bn = layers.BatchNormalization()  # BN
        # MLP 结构提取高教特征交互信息
        self.ffn = keras.Sequential([layers.Dense(i, activation=Dice() if ffn_activation != 'prelu' else layers.PReLU())
                                     for i in ffn_hidden_units])
        self.dropout = layers.Dropout(dnn_dropout)
        self.final_output = layers.Dense(1)


    def call(self, inputs, missing_val=0):
        # 将o作为类别的padding值，需要对特征使用label_enocde的时候从1开始进行
        # 需要事先对behavior中不等长的序列进行填充为等长，
        # inputs的输入是不考虑batch维度的
        # mask [seq_len]
        dense_user_feat, sparse_user_feat, dense_item_feat, sparse_item_feat, behavior_feat = inputs
        mask = tf.ones_like(behavior_feat, dtype=tf.float32)
        mask = tf.where(tf.equal(behavior_feat, missing_val), 0.0, mask) # 将pad的进行mask
        print(mask.shape)
        if self.user_sparse_feature_index:
            embed_user_feat = tf.concat([
                self.embed_layers['embed_'+k](sparse_user_feat[:, i])
                for k, i in self.user_sparse_feature_index.items()
            ], axis=-1)  # [batch, user_embed_dims]
        else: # 如果没有用户的类别特征会传入一个全为0的特征来替代embedding向量
            embed_user_feat = sparse_user_feat
            
        user_embed = tf.concat([dense_user_feat, embed_user_feat], axis=-1)  # [batch, user_tot_dims]

            
        if self.item_sparse_feature_index.items:    
            embed_item_feat = tf.concat([
                self.embed_layers['embed_'+k](sparse_item_feat[:, i])
                for k, i in self.item_sparse_feature_index.items()
            ], axis=-1)  # [batch, item_embed_dims]
        else:
            embed_item_feat = sparse_item_feat
            
        item_embed = tf.concat([dense_item_feat, embed_item_feat], axis=-1)  # [batch, item_tot_dims]
        
        
        embed_behaviors = tf.concat([
            # 使用物品的embedding向量来得到用户的历史embedding序列
            self.embed_layers['embed_{v[0]}_{v[1]}_{v[3]}'.format(v=key.split('_'))](behavior_feat[:, i])
            for key, i in self.behavior_feature_index.items()
        ], axis=-1)  # [batch, tot_seq_embed_dims]

        embed_behaviors = tf.reshape(embed_behaviors, shape=(-1, self.maxlen, embed_item_feat.shape[1]))  # [batch, seq_len, item_embed_dims]
        # 原本的mask是对时间点上的两个特征都进行了padding=0，所以shape = [None, 2*seq_len] 
        # 在attention中我们需要知道的是time时间点是否是被 padding的，也就是两个特征有一个被pad=0就代表这个时间点需要被mask（即 i与i+1 都是表征 i//2 这个点的特征，i为偶数）
        # reshape采用了按行填充的方式，所以这里可以使用间隔索引乘积的方式得到真实的mask序列 [None, seq_len]
        # trick!
        mask = tf.gather(mask, indices=list(range(0, mask.shape[1], 2)), axis=-1) *  tf.gather(mask, indices=list(range(1, mask.shape[1], 2)), axis=-1)# [batch, seq_len]
        print(mask.shape, embed_behaviors.shape)
        behavior_out = self.attention([embed_item_feat, embed_behaviors, mask]) # [batch, items_embedding_dims]
        print('behavior,', behavior_out.shape)
        tot_feat = tf.concat([user_embed, item_embed, behavior_out], axis=-1) # [batch, feat_dims]
        tot_feat = self.bn(tot_feat)
        out = self.ffn(tot_feat)
        out = self.dropout(out)
        out = self.final_output(out)
        print(out.shape)
        return tf.nn.sigmoid(out) # [batch, 1]

    def build_graph(self, dims=[5, 3, 5, 3, 3]):
        d_u, s_u, d_i, s_i, k = dims
        # 其中 k 代表用户行为序列中一个时间点上的特征数目
        dense_user_input = keras.Input(shape=(d_u,), dtype=tf.float32)
        sparse_user_input = keras.Input(shape=(s_u,), dtype=tf.float32)
        dense_item_input = keras.Input(shape=(d_i,), dtype=tf.float32)
        sparse_item_input = keras.Input(shape=(s_i,), dtype=tf.float32)
        behavior_input = keras.Input(shape=(k*self.maxlen,), dtype=tf.float32)

        model = keras.Model(inputs=[dense_user_input, sparse_user_input, dense_item_input, sparse_item_input, behavior_input],
                            outputs=self.call([dense_user_input, sparse_user_input, dense_item_input, sparse_item_input, behavior_input]))

        return model



In [925]:
att_hidden_units = [80, 40]
ffn_hidden_units = [256, 128, 64]
dnn_dropout = 0.5
att_activation = 'sigmoid'
ffn_activation = 'prelu'
din = DIN(sparse_dict, [user_sparse_index, item_sparse_index, behavior_index], att_hidden_units, ffn_hidden_units, att_activation,
                    ffn_activation, maxlen, dnn_dropout)

keras.backend.clear_session()

model = din.build_graph([1, 1, 1, 2, 2])

model.summary()
model.compile(loss=keras.losses.binary_crossentropy,
             optimizer=keras.optimizers.Adam(),
             metrics=[keras.metrics.AUC()])

model.fit(train_x, train['label'], epochs=10, batch_size=512, validation_data=(val_x, val['label']))
print(model.evaluate(test_x, test['label']))

(None, 20)
(None, 10) (None, 10, 16)
behavior, (None, 16)
(None, 1)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 tf.__operators__.getitem (Slic  (None,)             0           ['input_4[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 tf.__operators__.getitem_

1120/1120 [==============================] - 23s 19ms/step - loss: 0.6109 - auc: 0.7308 - val_loss: 0.5588 - val_auc: 0.7969
Epoch 2/10
1120/1120 [==============================] - 17s 16ms/step - loss: 0.5055 - auc: 0.8445 - val_loss: 0.5102 - val_auc: 0.8438
Epoch 3/10
1120/1120 [==============================] - 17s 16ms/step - loss: 0.4701 - auc: 0.8717 - val_loss: 0.5033 - val_auc: 0.8520
Epoch 4/10
1120/1120 [==============================] - 18s 16ms/step - loss: 0.4565 - auc: 0.8821 - val_loss: 0.5040 - val_auc: 0.8561
Epoch 5/10
1120/1120 [==============================] - 17s 16ms/step - loss: 0.4469 - auc: 0.8891 - val_loss: 0.5112 - val_auc: 0.8524
Epoch 6/10
1120/1120 [==============================] - 17s 15ms/step - loss: 0.4388 - auc: 0.8949 - val_loss: 0.5114 - val_auc: 0.8537
Epoch 7/10
1120/1120 [==============================] - 18s 16ms/step - loss: 0.4320 - auc: 0.8996 - val_loss: 0.5155 - val_auc: 0.8549
Epoch 8/10
1120/1120 [==============================] - 18s

# YoutubeDNN

### 数据规整函数

In [16]:

import random
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences

def gen_data_set(data, negsample=0):

    data.sort_values("timestamp", inplace=True)
    item_ids = data['movie_id'].unique()

    train_set = []
    test_set = []
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['movie_id'].tolist()
        rating_list = hist['rating'].tolist()

        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            if i != len(pos_list) - 1:
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1, len(hist[::-1]), rating_list[i]))
                for negi in range(negsample):
                    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1]), 0)) # 负采样样本看作评分为0
            else:
                test_set.append((reviewerID, hist[::-1], pos_list[i], 1,len(hist[::-1]),rating_list[i]))

    random.shuffle(train_set)
    random.shuffle(test_set)

    print(len(train_set[0]),len(test_set[0]))

    return train_set,test_set

def gen_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {"user_id": train_uid, "movie_id": train_iid, "hist_movie_id": train_seq_pad,
                         "hist_len": train_hist_len}

    for key in ["gender", "age", "occupation", "zip"]:
        train_model_input[key] = user_profile.loc[train_model_input['user_id']][key].values

    return train_model_input, train_label

In [228]:
data_path = "../data/"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames, engine='python')
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames, engine='python')
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames, engine='python')

data = pd.merge(pd.merge(ratings,movies),user).iloc[:1000000]

In [106]:
data

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
59995,850,1029,4,975351703,Dumbo (1941),Animation|Children's|Musical,M,35,0,60640
59996,850,1207,5,975349928,To Kill a Mockingbird (1962),Drama,M,35,0,60640
59997,850,608,4,975350080,Fargo (1996),Crime|Drama|Thriller,M,35,0,60640
59998,850,2194,3,975357193,"Untouchables, The (1987)",Action|Crime|Drama,M,35,0,60640


In [229]:
#data = pd.read_csvdata = pd.read_csv("./movielens_sample.txt")
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", ]
SEQ_LEN = 50
negsample = 2

# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`

features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip']
feature_max_idx = {}
for feature in features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # 将0作为mask
    if feature == 'user_id':
        user_idx_2_rawid = dict(zip(lbe.transform(lbe.classes_)+1, lbe.classes_))
    elif feature == 'movie_id':
        doc_idx_2_rawid = dict(zip(lbe.transform(lbe.classes_)+1, lbe.classes_))
        
    feature_max_idx[feature] = data[feature].max() + 1

user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')

item_profile = data[["movie_id"]].drop_duplicates('movie_id')

user_profile.set_index("user_id", inplace=True)

user_item_list = data.groupby("user_id")['movie_id'].apply(list)

train_set, test_set = gen_data_set(data, negsample)

train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

# 2.count #unique features for each sparse field and generate feature config for sequence feature



100%|█████████████████████████████████████████████████████████████████████████████| 6034/6034 [00:28<00:00, 215.32it/s]


6 6


In [230]:
user_feat_columns = ['user_id', 'gender', 'age', 'occupation', 'zip']
item_feat_columns = ['movie_id']
user_behavior_columns = ['hist_movie_id']


def get_array(dict_input, label, columns):
    user_columns = np.transpose(np.stack([dict_input[i]
                            for i in columns
                            ], axis=0))
    movie_id = np.expand_dims(dict_input['movie_id'], axis=1) # 点击的商品
    print(user_columns.shape, label.shape, movie_id.shape)
    return user_columns, label, movie_id

train_user_sparse_feat, train_label, target_movie = get_array(train_model_input, train_label, user_feat_columns)
train_hist_movie_feat = train_model_input['hist_movie_id']

(2963796, 5) (2963796,) (2963796, 1)


In [231]:
from tensorflow import keras

class DNN(keras.layers.Layer):
    """
    dnn 部分
    """
    def __init__(self, hidden_units, activation='relu', reg_l2=0.01,**kwgs):
        super().__init__(**kwgs)
        self.dnn_network = [keras.layers.Dense(i, activation=activation, 
                                               kernel_regularizer=keras.regularizers.l2(reg_l2),  
                                               bias_regularizer=keras.regularizers.l2(reg_l2)) for i in hidden_units] # 使用正则化可以防止过拟合
    
    def call(self, inputs):
        x = inputs
        for layer in self.dnn_network:
            x = layer(x)
        return x

# 用于计算 softmax 的负采样函数
class SampleSoftMax(keras.layers.Layer):
    def __init__(self, num_items, embed_dims, numsample):
        self.num_items = num_items
        self.embed_dims = embed_dims
        self.numsample= numsample
        super().__init__()
    
    def build(self, input_shape):
        self.items_w = self.add_weight(shape=[self.num_items, self.embed_dims],
                          initializer=tf.keras.initializers.random_normal,
                          trainable=True,
                          name='item_embedding') # embed
        self.bias_ = self.add_weight(shape=[self.num_items,],
                                    initializer=tf.keras.initializers.zeros,
                                    trainable=False, name='bias_0')
    
    def call(self, inputs):
        out_user_embed, target_movie_id = inputs
        loss = tf.nn.sampled_softmax_loss(
            weights=self.items_w,
            biases=self.bias_,
            labels=tf.reshape(tf.cast(target_movie_id, tf.int32), shape=[-1,1]), # 真实的正例索引
            inputs=out_user_embed,
            num_classes=self.num_items,
            num_sampled=self.numsample,
            num_true=1
        )
        return tf.expand_dims(loss, 1)  # [batch, 1]
        
    
    def get_embeding(self, indx):
        return tf.nn.embedding_lookup(self.items_w, tf.cast(indx, tf.int32))
    

class YoutubeDNN(keras.layers.Layer):
    def __init__(self, feature_max_idx, num_items, embed_dims, SEQ_LEN, dnn_hidden_units, dnn_activation, dnn_reg_l2, numsample=5):
        assert dnn_hidden_units[-1] == embed_dims, '用户embedding向量要与video embedding维度一致'
        self.user_feat_columns = user_feat_columns
        self.seq_len = SEQ_LEN

        self.embedding_layers = {
            k: keras.layers.Embedding(v, embed_dims, name=k+'_embedding')
            for k, v in feature_max_idx.items()
        }
        self.num_items = num_items
        self.embed_dims = embed_dims
        self.numsample = 5
        self.dnn = DNN(dnn_hidden_units, dnn_activation, dnn_reg_l2)
        self.sampleSoftMax = SampleSoftMax(num_items, embed_dims, numsample)
        super().__init__()


    def call(self, inputs):
        user_sparse_feat, hist_movie_feat, target_movie_id = inputs
        embed_user = tf.concat([
            self.embedding_layers[k](user_sparse_feat[:, i])
            for i, k in enumerate(self.user_feat_columns)
        ], axis=-1) # [batch, embed_dims_tot]
        mask = tf.where(tf.equal(hist_movie_feat, 0), 1.0, 0.0) # [batch, seq_len]
        mask_count = tf.reduce_sum(mask, axis=1, keepdims=True) # [batch,1] 
        hist = self.embedding_layers['movie_id'](hist_movie_feat) # [batch, seq_len, embed_dims]
        mask_h = tf.expand_dims(mask, 1)# [batch, 1, seq_len]
        video_embed = tf.matmul(mask_h, hist) # [batch, 1, embed_dims]
        video_embed = tf.squeeze(video_embed, axis=1) # [batch, embed_dims]  sum 池化
        video_embed = video_embed / (0.001 + mask_count) # 平均池化
        
        print(mask.shape, video_embed.shape, mask_count.shape)
        x = tf.concat([embed_user, video_embed], axis=-1) #[batch, input_dims]

        self.user_embedding = out_user_embed = self.dnn(x) # [batch, embed_dims]

        return self.sampleSoftMax([out_user_embed, target_movie_id]) # 返回的是loss


    def build_graph(self):
        user_sparse_feat = keras.Input(shape=(5,), dtype=tf.float32, name='input_user')
        hist_movie_feat = keras.Input(shape=(self.seq_len,), dtype=tf.float32, name='input_hist_video')
        target_movie_id = keras.Input(shape=(1,), dtype=tf.float32, name='click_movie')

        inputs = [user_sparse_feat, hist_movie_feat, target_movie_id]
        outputs = self.call(inputs)
        model = keras.Model(inputs=inputs, outputs=outputs)
        
        model.__setattr__('user_input', [user_sparse_feat, hist_movie_feat])
        model.__setattr__('user_embedding', self.user_embedding)
        model.__setattr__('item_input', target_movie_id)
        model.__setattr__('item_embedding', self.sampleSoftMax.get_embeding(model.item_input))
        
        # 添加一些列
        return model

In [232]:
num_items = feature_max_idx['movie_id']
embed_dims = 60
SEQ_LEN = SEQ_LEN
dnn_hidden_units = (256, 128, embed_dims)
dnn_activation = 'relu'
dnn_reg_l2 = 0.0001

model = YoutubeDNN(
    feature_max_idx, num_items, embed_dims, SEQ_LEN, dnn_hidden_units, dnn_activation, dnn_reg_l2, numsample=5
).build_graph()

(None, 50) (None, 60) (None, 1)


In [233]:
def loss_obj(ytrue, y_pred): # 自定义 loss 
    return tf.reduce_mean(y_pred)

model.compile(optimizer=keras.optimizers.Adam(), loss=loss_obj)
model.fit([train_user_sparse_feat, train_hist_movie_feat, train_label], train_label, epochs=10, batch_size=512, validation_split=0.1)

Epoch 1/10
5210/5210 [==============================] - 120s 23ms/step - loss: 0.2533 - val_loss: 0.1834
Epoch 2/10
5210/5210 [==============================] - 129s 25ms/step - loss: 0.1986 - val_loss: 0.1997
Epoch 3/10
5210/5210 [==============================] - 136s 26ms/step - loss: 0.1966 - val_loss: 0.1847
Epoch 4/10
5210/5210 [==============================] - 135s 26ms/step - loss: 0.1942 - val_loss: 0.1899
Epoch 5/10
5210/5210 [==============================] - 136s 26ms/step - loss: 0.1930 - val_loss: 0.1944
Epoch 6/10
5210/5210 [==============================] - 203s 39ms/step - loss: 0.1946 - val_loss: 0.1874
Epoch 7/10
5210/5210 [==============================] - 271s 52ms/step - loss: 0.1976 - val_loss: 0.1795
Epoch 8/10
5210/5210 [==============================] - 146s 28ms/step - loss: 0.1948 - val_loss: 0.1860
Epoch 9/10
5210/5210 [==============================] - 149s 29ms/step - loss: 0.1922 - val_loss: 0.1887
Epoch 10/10
5210/5210 [==============================] 

### 效果评估

In [234]:

model_item_embded = keras.Model(inputs=model.item_input, outputs=model.item_embedding)
model_user_embed = keras.Model(inputs=model.user_input, outputs=model.user_embedding)

items_input = np.arange(1, num_items) # 0 是掩码位置不进行考虑
doc_embs = tf.squeeze(model_item_embded(items_input)) 
doc_embs = doc_embs / tf.linalg.norm(doc_embs, axis=1, keepdims=True) # 所有item物品的embedding 进行归一化


In [235]:
test_user_sparse_feat, test_label, test_click_movie_id = get_array(test_model_input, test_label, user_feat_columns)
test_hist_movie_feat = test_model_input['hist_movie_id']
user_embs = model_user_embed([test_user_sparse_feat, test_hist_movie_feat])
user_embs = user_embs / tf.linalg.norm(user_embs, axis=1, keepdims=True) # 测试集所有用户的embedding
user_embs.shape

(6034, 5) (6034,) (6034, 1)


TensorShape([6034, 60])

In [236]:
# 模拟server的阶段，使用LSH来找到和用户embedding最近的movie进行推荐
from annoy import AnnoyIndex
import collections
import pickle

def get_youtube_recall_res(user_embs, doc_embs, user_idx_2_rawid, doc_idx_2_rawid, topk):
    """近邻检索，这里用annoy tree"""
    # 把doc_embs构建成索引树
    f = user_embs.shape[1]
    t = AnnoyIndex(f, 'angular')
    for i, v in enumerate(doc_embs):
        t.add_item(i, v)
    t.build(10)
    # 可以保存该索引树 t.save('annoy.ann')
    
    # 每个用户向量， 返回最近的TopK个item
    user_recall_items_dict = collections.defaultdict(dict)
    for i, u in enumerate(user_embs):
        recall_doc_scores = t.get_nns_by_vector(u, topk, include_distances=True)
        # recall_doc_scores是(([doc_idx], [scores]))， 这里需要转成原始doc的id
        raw_doc_scores = list(recall_doc_scores)
        raw_doc_scores[0] = [doc_idx_2_rawid[i+1] for i in raw_doc_scores[0]] # 此处 doc_idx + 1来忽略 mask 的位置的影响
        # 转换成实际用户id
        try:
            user_recall_items_dict[user_idx_2_rawid[i]] = dict(zip(*raw_doc_scores))
        except:
            continue
    
    # 默认是分数从小到大排的序， 这里要从大到小
    user_recall_items_dict = {k: sorted(v.items(), key=lambda x: x[1], reverse=True) for k, v in user_recall_items_dict.items()}
    
    # 保存一份
    pickle.dump(user_recall_items_dict, open('youtube_u2i_dict.pkl', 'wb'))
    
    return user_recall_items_dict


user_recall_items_dict = get_youtube_recall_res(user_embs, doc_embs, user_idx_2_rawid, doc_idx_2_rawid, topk=100)

In [237]:
last_click_item_dict = {
    user_idx_2_rawid[i]: doc_idx_2_rawid[j]
    for i, j in zip(test_user_sparse_feat[:, 0], np.squeeze(test_click_movie_id))
} # 记录用户最后点击的真实movie_id 是多少

In [238]:
# 依次评估召回的前10, 20, 30, 40, 50个文章中的击中率
def metrics_recall(user_recall_items_dict, last_click_item_dict, topk=100):
    user_num = len(user_recall_items_dict)
    
    for k in range(20, topk+1, 20):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            if user in last_click_item_dict:
                # 获取前k个召回的结果
                tmp_recall_items = [x[0] for x in user_recall_items_dict[user][:k]]
                if last_click_item_dict[user] in set(tmp_recall_items):
                    hit_num += 1
        
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)


metrics_recall(user_recall_items_dict, last_click_item_dict, 100)

 topk:  20  :  hit_num:  23 hit_rate:  0.00381 user_num :  6033
 topk:  40  :  hit_num:  94 hit_rate:  0.01558 user_num :  6033
 topk:  60  :  hit_num:  137 hit_rate:  0.02271 user_num :  6033
 topk:  80  :  hit_num:  226 hit_rate:  0.03746 user_num :  6033
 topk:  100  :  hit_num:  295 hit_rate:  0.0489 user_num :  6033


# DIEN

### 实现AUGRU

In [294]:
_BIAS_VARIABLE_NAME = "bias"

_WEIGHTS_VARIABLE_NAME = "kernel"

class AUGRUCell(keras.layers.Layer):
    """
    实现AUGRU
    """
    def __init__(self,
                 num_units,
                 activation=tf.math.tanh,
                 reuse=None, # 控制variable_scope的名字是否可以复用的，在同一个scope下的变量可以通过tf.get_variable来获取，从而实现共享变量
                 kernel_initializer=keras.initializers.random_normal,
                 bias_initializer=keras.initializers.Zeros):

        super(AUGRUCell, self).__init__()
        self._num_units = num_units
        self._activation = activation or math_ops.tanh
        self._kernel_initializer = kernel_initializer
        self._bias_initializer = bias_initializer
        self._gate_linear = None
        self._candidate_linear = None

    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units
        
    def build(self, input_shape):
        embed_dims = input_shape[-1] - 1 # trick: -1 是因为最后一个维度作为了储存注意力分数
        with tf.name_scope('gates'):    
            self.w1 = self.add_weight(shape=(embed_dims+self._num_units, 2*self._num_units),
                                     initializer=self._kernel_initializer,
                                     trainable=True,
                                     name=_WEIGHTS_VARIABLE_NAME)
            self.bias1 = self.add_weight(shape=(2*self._num_units,),
                                        initializer=self._bias_initializer,
                                        trainable=True,
                                        name=_BIAS_VARIABLE_NAME
                                        )
            
        with tf.name_scope('candidate'):
            self.w2 = self.add_weight(shape=(embed_dims+self._num_units, self._num_units),
                                     initializer=self._kernel_initializer,
                                     trainable=True,
                                     name=_WEIGHTS_VARIABLE_NAME)
            self.bias2 = self.add_weight(shape=(self._num_units,),
                                        initializer=self._bias_initializer,
                                        trainable=True,
                                        name=_BIAS_VARIABLE_NAME
                                        )
            
        
    def __call__(self, inputs, state):
        return self.call(inputs, state)

    def call(self, inputs, state):
        """Gated recurrent unit (GRU) with nunits cells.
        trick: 将score merge 到inputs 的最后一列，表示该时期的注意力得分
        params:
            inputs: [batch, embedims + 1] # 第 t 时期的输入， 其中1是注意力得分
            state: ([batch, num_units], )  传入的是元组，第 t-1 期的cell状态
  Call arguments:
    inputs: A 2D tensor.
    states: List of state tensors corresponding to the previous timestep.
    training: Python boolean indicating whether the layer should behave in
      training mode or in inference mode. Only relevant when `dropout` or
      `recurrent_dropout` is used.
        """
        state = state[0] # status 是元组类型，要通过索引才能拿道具体的state
        item_embed_dims = inputs.shape[1] - 1
        inputs, att_score = tf.split(inputs, [item_embed_dims, 1], axis=-1) # [batch, embed_dims], [batch, 1]
        x = tf.concat([inputs, state], axis=-1) # [batch, embed_dims + num_units]   
        value = tf.sigmoid(tf.matmul(x, self.w1) + self.bias1) # [batch, 2*num_units]
        
        r, u = tf.split(value, num_or_size_splits=2, axis=1) # 2*[batch, num_units]

        r_state = r * state # [batch, num_units] 
        c = self._activation(tf.matmul(x, self.w2) + self.bias2)  # [batch, num_units] 
        print(u.shape, c.shape, att_score.shape)
        u = (1.0 - att_score) * u
        new_h = u * state + (1 - u) * c
        
        return new_h, new_h
    
### for test
# batch_size = 10
# embed_dims = 129
# output_dim = 120
# num_units = 20

# inputs = tf.Variable(tf.random.normal([batch_size, 2, embed_dims]))

# gruCell = tf.keras.layers.RNN(AUGRUCell(num_units), return_sequences=True, return_state=True)
# print('model after build!', inputs.shape)
# output, state = gruCell(inputs)



### AttentionPoolingLayer
#### 复用DIN的注意得分单元

In [454]:
# 继续使用了DIN中的注意力机制来得到注意力分数
# Dice自适应激活函数
from keras import layers

class Dice(layers.Layer):
    def __init__(self):
        super().__init__()
        self.alpha = self.add_weight(shape=(), initializer=keras.initializers.Zeros(), dtype=tf.float32)
        self.bn = layers.BatchNormalization(center=False, scale=False, trainable=True) # 只使用BN的归一化，不学习移动和缩放参数

        
    def call(self, x):
        flag = False # 是否是由于负采样引起的升维情况
        if len(x.shape) == 4:
            flag = True
            seq_len, neg_sample, embed_dims = x.shape[1:]
            x = tf.reshape(x, shape=[-1, neg_sample, embed_dims])
        p = tf.nn.sigmoid(self.bn(x))
        if flag == True:
            x = tf.reshape(x, shape=[-1, seq_len, neg_sample, embed_dims])
        return p*x + (1-p)*self.alpha*x


class AttentionPoolingLayer(layers.Layer):
    """
    注意力层，用于加权融合用户历史行为序列
    """
    def __init__(self, att_hidden_units, activation='prelu', return_score=False, **kwgs):
        super().__init__(**kwgs)
        self.att_dense = [layers.Dense(unit, activation=Dice() if activation != 'prelu' else layers.PReLU()) for unit in att_hidden_units]
        self.att_final_dense = layers.Dense(1)
        self.return_score = return_score

    def call(self, inputs):

        """
        inputs [query_items, hist_items, mask]
        quer_items (q): [batch, seq_len, items_embedding_dims]
        hist_items (k): [batch, seq_len, items_embedding_dims]
        mask : [batch, 1] 传入每个序列的真实长度
        """
        q, k, mask = inputs
        mask = tf.sequence_mask(mask, k.shape[1], dtype=tf.float32) # [batch, 1, seq_len]
        mask = tf.squeeze(mask, axis=1) # [batch, seq_len]
        val = tf.identity(k) # copy
        print('1', q.shape, k.shape, mask.shape)
        
        x = tf.concat([q, k, q-k, q*k], axis=-1)  # [batch, seq_len, 4*items_embedding_dims]
        for layer in self.att_dense:
            x = layer(x)
        # x shape 为 [batch, seq_len, last_hidden_units]
        w = self.att_final_dense(x)  # [batch, seq_len, 1]
        w = tf.squeeze(w, axis=-1)  # [batch, seq_len]

        w = tf.where(tf.equal(mask, 0), -(1<<31)*1.0, w)  # [batch, seq_len] 将mask位置的权重调整为很大的负数，在经过softmax之后权重会变为0

        # 使用 softmax 归一化权重
        w = tf.nn.softmax(w, axis=-1)  # [batch, seq_len]
        if self.return_score: return w # 返会注意力得分，后续融入AUGRU中
        
        w = tf.expand_dims(w, axis=1)  # [batch, 1, seq_len] 用户行为序列与目标query的 score
        # 不需要返回score 直接进行DIN中的加权池化
        outputs = tf.matmul(w, val)  # [batch, 1, items_embedding_dims]
        outputs = tf.squeeze(outputs, axis=1)  # [batch, items_embedding_dims] # 历史行为序列的池化embedding
            
        return outputs 


### 构建兴趣进化层

In [711]:
class DNN(keras.layers.Layer):
    """
    dnn 部分
    """
    def __init__(self, hidden_units, activation='prelu', reg_l2=0.01, **kwgs):   
        self.dnn_network = [keras.layers.Dense(i, activation=layers.PReLU() if activation == 'prelu' 
                                               else (Dice() if isinstance(activation, Dice) else activation),  # 对每一层确定一个激活函数
                                               kernel_regularizer=keras.regularizers.l2(reg_l2),  
                                               bias_regularizer=keras.regularizers.l2(reg_l2)) for i in hidden_units] # 使用正则化可以防止过拟合
        super().__init__(**kwgs)
        
    def call(self, inputs):
        x = inputs
        for layer in self.dnn_network:
            x = layer(x)
        return x

    
class DIEN(keras.layers.Layer):
    def __init__(self, sparse_feature_dict, sparse_feat_index_list, num_units=20, max_len=20, neg_sample=2,
                 aux_hidden_units=(128, 64, 1), aux_activation='sigmoid', att_hidden_units=(200, 60), att_activation=Dice(),
                 return_score=True, dnn_hidden_units=(200, 80), dnn_activation='prelu', embed_reg=0.001,
                dropout=0):
        """
        params:
            sparse_feature_dict: 记录了需要embedding的特征信息
            sparse_feat_index_lost : 记录各种类型的叙述特征和输入的索引之间的映射关系
        """
        super().__init__()
        self.max_len = max_len
        self.neg_sample = neg_sample
        self.aux_dnn = DNN(aux_hidden_units, activation=aux_activation, name='for_aux') # 用于学习计算兴趣和目标广告相似度的dnn结构
        self.last_dnn = DNN(dnn_hidden_units, dnn_activation, name='last_dnn_interaction') # 最后的输出dnn
        self.att = AttentionPoolingLayer(att_hidden_units, att_activation, return_score=return_score, name='attention_layer')
        self.extract_int = layers.GRU(num_units, return_sequences=True)
        self.evolution_int = tf.keras.layers.RNN(AUGRUCell(num_units))
        self.embed_layers = {
            k: layers.Embedding(v[0], v[1], embeddings_regularizer=keras.regularizers.l2(embed_reg), name=k)
            for k, v in sparse_feature_dict.items()
        }
        self.user_sparse_feat, self.hist_sparse_feat, self.item_sparse_feat = sparse_feat_index_list
        self.dropout = layers.Dropout(dropout)
        self.ctr = layers.Dense(1, activation='sigmoid', name='ctr')
        self.bn = layers.BatchNormalization()  # 使用 DICE 函数就不需要BN了
        
    
    def auxiliary_loss(self, is_pos, state_seq, click_seq, no_click_seq, mask):
        """
        is_pos 代表抽取出来的是否是正例 [None, 1]
        注意：在序列模型中使用 click_t+1 来作为label，计算和h_t的损失，表明有损失值序列长度为原seq_length-1，因为最后一个时间步的输出就没法计算了
        k = seq_len - 1
        dnn: 用来计算相似度的dnn结构
        state_seq:  [None, k, num_units]
        click_seq: [None, k, embed_dims]
        no_click_seq: 负采样的点 [None, k, neg_sample, embed_dims]
        mask: 真实的序列长度 [None, 1]
        """
        max_len = click_seq.shape[1]
        neg_sample = no_click_seq.shape[-2]
        num_units = state_seq.shape[-1]
        mask = tf.sequence_mask(mask, max_len, dtype=tf.float32)[:, 0, :] # [None, k]

        x_pos = tf.concat([state_seq, click_seq], axis=-1) # [None, k, num_units + embed_dims]
        # 只对将来点击的正例进行学习，对于负例不计算损失
#         print('p_pos', p_pos.shape)
        p_pos = self.aux_dnn(x_pos) # [None, k, 1]
        is_ok_mask = tf.reshape(tf.tile(is_pos, [1, x_pos.shape[1]]), shape=(-1, p_pos.shape[1], p_pos.shape[2]))
        print(is_ok_mask.shape)
        p_pos = tf.where(tf.math.greater(is_ok_mask, 0), p_pos, 1) # loss 只计算兴趣状态与真实点击的物品embedding之间的loss
        loss_pos = -tf.math.log(p_pos)

        status_neg = tf.tile(state_seq, [1, 1, neg_sample]) # [None, k, neg_samples * num_units]
        status_neg = tf.reshape(status_neg, shape=(-1, status_neg.shape[1], neg_sample, num_units)) # [None, k, neg_samples, num_units]
        x_neg = tf.concat([status_neg, no_click_seq], axis=-1) # [None, k, neg_samples, num_units + embed_dims]
        p_neg = tf.squeeze(self.aux_dnn(x_neg), axis=-1) # [None, k, neg_samples]
        loss_neg = -tf.math.log(1 - p_neg) # [None, k, neg_samples]
        loss_neg = tf.reduce_mean(loss_neg, axis=-1, keepdims=True) # [None, k, 1]
        
        res = tf.reduce_mean(loss_pos + loss_neg) 
        return tf.reduce_mean(loss_pos + loss_neg) # 标量

    
    def interest_evolution(self, is_pos, hist_seq, pos_query, neg_query, seq_len):
        """
        整合构建DIEN中的兴趣进化网络
        params:
            is_pos [None, 1] # pos_query 是否是点击的物品，还是采样出来的负例
            hist_seq [None, seq_len, embed_dims]
            pos_query: [None, embed_dims]
            neg_query: [None, neg_sample, embed_dims]
            seq_len: [None, 1]
        """
        print(hist_seq.shape, pos_query.shape, neg_query.shape, seq_len.shape)
        aux_loss = 0
        embed_dims, length = pos_query.shape[-1], hist_seq.shape[1]
        pos_query = tf.tile(pos_query, [1, hist_seq.shape[1]]) # [None, seq_len*embed_dims]
        pos_query = tf.reshape(pos_query, [-1, hist_seq.shape[1], embed_dims]) # [None, seq_len, embed_dims]
        
        neg_query = tf.tile(neg_query, [1, 1, length]) #[None, neg_sample, seq_len*embed_dims]
        neg_query = tf.reshape(neg_query, [-1, neg_query.shape[1], length, embed_dims]) # [None, neg_sample, seq_len, embed_dims]
        neg_query = tf.transpose(neg_query, [0, 2, 1, 3]) # [None, seq_len, neg_sample, embed_dims]
        
        # 兴趣提取层
        state_seq = self.extract_int(hist_seq) # 抽取出兴趣embedding [None, seq_len, num_units]
        aux_loss = self.auxiliary_loss(is_pos,
                                      state_seq[:,:-1,:], # 抽取出 seq_len - 1个兴趣状态
                                      hist_seq[:,1:,:],
                                      neg_query[:,1:,:,:],
                                      seq_len - 1.0 # 序列长度为 seq_len - 1
                                      )
        # 兴趣进化层
        score = self.att([pos_query, state_seq, seq_len]) # [None, seq_len]
        # 拼接到RNN的输入中作为输入
        score = tf.expand_dims(score, axis=-1)# [None, seq_len, 1]
        state_seq_with_score = tf.concat([state_seq, score], axis=-1) # [None, seq_len ,num_units+1]
        out = self.evolution_int(state_seq_with_score) # [None, num_units]
        return out, aux_loss
    
    
    def call(self, inputs):
        """
        inputs： [user_sparse_feat, user_hist_sparse_feat, pos_query_item, neg_query_item, seq_len]
        user_sparse_feat: [None, feat_kinds]
        user_hist_sparse_feat: [None, max_seq, k]  # k 表示的是一个时间点有k个特征
        pos_query_item: [None, j] # 表示的是目标广告的类别有 j 个
        neg_query_item: [None, m, j] # 表示的是负采样目标广告的类别有 j 个，1 pos vs m neg
        seq_len: [None, 1] # 序列长度
        is_pos: 点击的物品是否是正例
        """
        user_sparse_feat, user_hist_sparse_feat, pos_query_item, neg_query_item, seq_len, is_pos = inputs
        embed_user_feat = tf.concat([
            self.embed_layers[k](user_sparse_feat[:, v])
            for k, v in self.user_sparse_feat.items()
        ], axis=-1) # [None, tot_user_embeds]
        embed_hist = tf.concat([
            self.embed_layers[k](user_hist_sparse_feat[:,:,i])
            for k, i in self.hist_sparse_feat.items()
        ], axis=-1) # [None, max_seq, k*embedding]
        pos_embed_items = tf.concat([
            self.embed_layers[k](pos_query_item[:, i])
            for k, i in self.item_sparse_feat.items()
        ], axis=-1) # [None, j*item_embeddings]
        neg_embed_items = tf.concat([
            self.embed_layers[k](neg_query_item[:, :, i])
            for k, i in self.item_sparse_feat.items()
        ], axis=-1) # [None, m, j*item_embeddings]
        
        # x [None, num_units]
        # 传入is_pos 和 label 是对应的，主要用于兴趣提取层去学习兴趣的表达
        x, aux_loss = self.interest_evolution(is_pos, embed_hist, pos_embed_items, neg_embed_items, seq_len)
        print('112', aux_loss.shape, x.shape) #建立静态图
        # 如果有连续特征可以加在这里
        x = tf.concat([x, embed_user_feat, pos_embed_items], axis=-1) # [None, tot_embeddings]
        x = self.bn(x)
        print('111', x.shape, aux_loss.shape)
        output = self.last_dnn(x)
        output = self.dropout(output)
        
        #添加 aux_loss
        self.add_loss(aux_loss*2)
        return self.ctr(output)
    
def build_graph(sparse_dict, sparse_feat_index_list, max_len, neg_sample=2):
    dien = DIEN(sparse_dict, sparse_feat_index_list, max_len=maxlen, neg_sample=neg_sample)
    user_sparse_feat = keras.Input(shape=(1,), dtype=tf.float32, name='user_sparse_input')
    user_hist_sparse_feat = keras.Input(shape=(max_len, 2), dtype=tf.float32, name='user_list_input')
    pos_query_item = keras.Input(shape=(2,), dtype=tf.float32, name='pos_item_sparse_input')
    neg_query_item = keras.Input(shape=(neg_sample, 2), dtype=tf.float32, name='neg_item_sparse_input')
    seq_len = keras.Input(shape=(1,), dtype=tf.float32, name='seq_len')
    is_pos =  keras.Input(shape=(1,), dtype=tf.float32, name='is_pos')
    inputs = [user_sparse_feat, user_hist_sparse_feat, pos_query_item, neg_query_item, seq_len, is_pos]
    outputs = dien(inputs)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

### 加载亚马逊数据集

In [299]:
thr = 500000 # 采样的数据数目

# 数据处理部分
def to_df(file_path, thr=float('inf')):
    """
    转化为DataFrame结构
    :param file_path: 文件路径
    :return:
    """
    with open(file_path, 'r') as fin:
        df = {}
        i = 0
        for line in fin:
            df[i] = eval(line)
            i += 1
            if i >= thr: break #采样1w条
        df = pd.DataFrame.from_dict(df, orient='index')
        return df

def build_map(df, col_name):
    """
    制作一个映射，键为列名，值为序列数字
    :param df: reviews_df / meta_df
    :param col_name: 列名
    :return: 字典，键
    """
    key = sorted(df[col_name].unique().tolist())
    m = dict(zip(key, range(1, 1+len(key)))) # 从1开始进行label_encode, 0作为特殊的padding scalar
    df[col_name] = df[col_name].map(lambda x: m[x])
    return m, key

print('==========Data Preprocess Start============')
reviews_df = to_df('../data/Electronics_10.json', thr) # 只需要对 review 做采样， meta的数量级较小
meta_df = to_df('../data/meta_Electronics.json')

==========Data Preprocess Start============


In [300]:
# 只保留reviews文件中出现过的商品
# 'reviewerID', 'asin', 'unixReviewTime' 分别表示用户id，物品id，购物的时间
meta_df = meta_df[meta_df['asin'].isin(reviews_df['asin'].unique())]
meta_df = meta_df.reset_index(drop=True)
reviews_df = reviews_df[['reviewerID', 'asin', 'unixReviewTime']]
meta_df = meta_df[['asin', 'categories']]
meta_df.shape, reviews_df.shape
# 物品类别只保留最后一个
meta_df['categories'] = meta_df['categories'].map(lambda x: x[-1][-1])
# 高维类别数据的编号索引map
# meta_df文件的物品ID映射
asin_map, asin_key = build_map(meta_df, 'asin')
# meta_df文件物品种类映射
cate_map, cate_key = build_map(meta_df, 'categories') # 对meta_df 的 categories列 进行 label_encoder，并返回 map_dict 以及 encode_list 
# reviews_df文件的用户ID映射
revi_map, revi_key = build_map(reviews_df, 'reviewerID')

user_count, item_count, cate_count, example_count = \
    len(revi_map), len(asin_map), len(cate_map), reviews_df.shape[0]


In [301]:

# reviews_df文件物品id进行映射，并按照用户id、浏览时间进行排序，重置索引
reviews_df['asin'] = reviews_df['asin'].map(lambda x: asin_map[x])
reviews_df = reviews_df.sort_values(['reviewerID', 'unixReviewTime'])
reviews_df = reviews_df.reset_index(drop=True)

# 各个物品对应的类别
cate_list = np.array([0]+meta_df['categories'].to_list(), dtype='int32')

reviews_df.columns = ['user_id', 'item_id', 'time']



In [366]:

from tqdm import tqdm
import random

tot_items = set(meta_df['asin'].unique())
cate_dict = dict(meta_df.values)
def gen_neg(pos_list, neg_sample=2):
    temp = list(set(tot_items) - set(pos_list))
    res = random.choices(range(len(temp)), k=neg_sample)
    return [[temp[i], cate_dict[temp[i]]] for i in res] # [neg_sample, 2]

train_data, val_data, test_data = [], [], []

neg_list = list(tot_items - set(pos_list))
for user_id, hist in tqdm(reviews_df.groupby('user_id')):
    pos_list = hist['item_id'].tolist()

    
    hist = []
    for i in range(1, len(pos_list)):
        hist.append([pos_list[i - 1], cate_list[pos_list[i-1]]])
        hist_i = hist.copy()
        if i == len(pos_list) - 1:
            test_data.append([1, hist_i, [pos_list[i], cate_list[pos_list[i]]], gen_neg(pos_list[:i+1]), len(hist_i), 1]) # 历史行为，【正样本id,类别】，label
            neg_item = random.choice(neg_list)
            test_data.append([1, hist_i, [neg_item, cate_dict[neg_item]], gen_neg(pos_list[:i+1]), len(hist_i), 0])
            # test_data.append([hist_i, [pos_list[i]], 1])
            # test_data.append([hist_i, [neg_list[i]], 0])
        elif i == len(pos_list) - 2:
            val_data.append([1, hist_i, [pos_list[i], cate_list[pos_list[i]]], gen_neg(pos_list[:i+1]), len(hist_i), 1]) # 历史行为，【正样本id,类别】，label
            neg_item = random.choice(neg_list)
            val_data.append([1, hist_i, [neg_item, cate_dict[neg_item]], gen_neg(pos_list[:i+1]), len(hist_i), 0])
            # val_data.append([hist_i, [pos_list[i]], 1])
            # val_data.append([hist_i, [neg_list[i]], 0])
        else:
            train_data.append([1, hist_i, [pos_list[i], cate_list[pos_list[i]]], gen_neg(pos_list[:i+1]), len(hist_i), 1]) # 历史行为，【正样本id,类别】，label
            neg_item = random.choice(neg_list)
            train_data.append([1, hist_i, [neg_item, cate_dict[neg_item]], gen_neg(pos_list[:i+1]), len(hist_i), 0])
            # train_data.append([hist_i, [pos_list[i]], 1])
            # train_data.append([hist_i, [neg_list[i]], 0])


# shuffle
random.shuffle(train_data)
random.shuffle(val_data)
random.shuffle(test_data)



100%|████████████████████████████████████████████████████████████████████████████| 20247/20247 [03:37<00:00, 92.99it/s]


In [714]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen = 10
embed_dims = 10


# create dataframe
columns = ['user_id', 'hist', 'pos_query', 'neg_query', 'seq_len', 'label']
train = pd.DataFrame(train_data, columns=columns)
val = pd.DataFrame(val_data, columns=columns)
test = pd.DataFrame(test_data, columns=columns)

# if no dense or sparse features, can fill with 0
print('==================Padding===================')

sparse_dict = { # 配置 embedding层的信息
    'user_id': (user_count+1, embed_dims),
    'item_id': (item_count+1, embed_dims),
    'item_category': (cate_count+1, embed_dims)
}

user_sparse_index = {
    'user_id': 0
}

item_sparse_index = {
    'item_id': 0,
    'item_category': 1
}

behavior_index = {
    'item_id': 0,
    "item_category": 1
}

train_y = train['label']
val_y = val['label']
test_y = test['label']


# 如果没有dense 或者 稀疏 特征使用全 0 的tesor来代替
train_x = [np.array(train['user_id'].values.reshape(-1, 1)),
        tf.reshape(pad_sequences(train['hist'], maxlen=maxlen, value=0, padding='post', truncating='post'), 
                   shape=[len(train), maxlen, -1]), # 用0表示是填充的项,
        np.array(list(train['pos_query'])), #[batch, 2]
        np.array(list(train['neg_query'])), # [batch, 2, 2]
        np.array(train['seq_len'].values.reshape(-1, 1)), # [batch, 1],
        np.array(train['label'].values.reshape(-1, 1)) # [batch, 1],
       ]

val_x = [np.array(val['user_id'].values.reshape(-1, 1)),
        tf.reshape(pad_sequences(val['hist'], maxlen=maxlen, value=0, padding='post', truncating='post'), 
                   shape=[len(val), maxlen, -1]), # 用0表示是填充的项,
        np.array(list(val['pos_query'])), #[batch, 2]
        np.array(list(val['neg_query'])), # [batch, 2, 2]
        np.array(val['seq_len'].values.reshape(-1, 1)), # [batch, 1]
         np.array(val['label'].values.reshape(-1, 1)) # [batch, 1],
       ]

test_x = [np.array(test['user_id'].values.reshape(-1, 1)), 
        tf.reshape(pad_sequences(test['hist'], maxlen=maxlen, value=0, padding='post', truncating='post'), 
                   shape=[len(test), maxlen, -1]), # 用0表示是填充的项, #[batch, maxlen, 2]
        np.array(list(test['pos_query'])), #[batch, 2]
        np.array(list(test['neg_query'])), # [batch, 2, 2]
        np.array(test['seq_len'].values.reshape(-1, 1)), # [batch, 1]
        np.array(test['label'].values.reshape(-1, 1)) # [batch, 1],
       ]
print('============Data Preprocess End=============')

for i in test_x: print(i.shape)

==================Padding===================
============Data Preprocess End=============
(40494, 1)
(40494, 10, 2)
(40494, 2)
(40494, 2, 2)
(40494, 1)
(40494, 1)


In [717]:
keras.backend.clear_session()
model = build_graph(sparse_dict, sparse_feat_index_list, max_len=maxlen)

model.compile(optimizer=keras.optimizers.Adam(),
            loss=keras.losses.binary_crossentropy, 
              metrics=[keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()])
model.summary()

(None, 10, 20) (None, 20) (None, 2, 20) (None, 1)
(None, 9, 1)
1 (None, 10, 20) (None, 10, 20) (None, 10)
(None, 20) (None, 20) (None, 1)
(None, 20) (None, 20) (None, 1)
112 () (None, 20)
111 (None, 50) ()
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_sparse_input (InputLayer)  [(None, 1)]         0           []                               
                                                                                                  
 user_list_input (InputLayer)   [(None, 10, 2)]      0           []                               
                                                                                                  
 pos_item_sparse_input (InputLa  [(None, 2)]         0           []                               
 yer)                                                                                 

In [722]:
# 得到 y 的标签
model.fit(train_x, train_y, epochs=5, batch_size=512, validation_data=(val_x, val_y))

Epoch 1/5
1120/1120 [==============================] - 191s 170ms/step - loss: 2.3947 - precision: 0.7908 - recall: 0.8010 - auc: 0.8866 - val_loss: 2.4208 - val_precision: 0.7921 - val_recall: 0.7504 - val_auc: 0.8705
Epoch 2/5
1120/1120 [==============================] - 224s 200ms/step - loss: 2.3908 - precision: 0.7927 - recall: 0.8036 - auc: 0.8886 - val_loss: 2.4156 - val_precision: 0.7804 - val_recall: 0.7840 - val_auc: 0.8732
Epoch 3/5
1120/1120 [==============================] - 203s 181ms/step - loss: 2.3881 - precision: 0.7941 - recall: 0.8051 - auc: 0.8897 - val_loss: 2.4148 - val_precision: 0.7964 - val_recall: 0.7578 - val_auc: 0.8743
Epoch 4/5
1120/1120 [==============================] - 206s 184ms/step - loss: 2.3865 - precision: 0.7939 - recall: 0.8077 - auc: 0.8907 - val_loss: 2.4123 - val_precision: 0.7878 - val_recall: 0.7740 - val_auc: 0.8759
Epoch 5/5
1120/1120 [==============================] - 215s 192ms/step - loss: 2.3852 - precision: 0.7951 - recall: 0.8070 -

In [739]:
model.evaluate(test_x, test_y)

1266/1266 [==============================] - 15s 12ms/step - loss: 2.4467 - precision: 0.7846 - recall: 0.7373 - auc: 0.8569


[2.446676015853882, 0.7846105098724365, 0.7372944355010986, 0.8568750619888306]

# FIBINET
## 1. SENET用于动态得到特征重要性
## 2. 双线性用于更细粒度的特征交互

In [843]:
import tensorflow as tf
import tensorflow.keras as keras
from keras import layers
import itertools


class DNN(layers.Layer):
    def __init__(self, units, activation='relu', dropout=0.3, reg_l2=0.01):
        super().__init__()
        self.layers = [layers.Dense(i, activation=activation, kernel_regularizer=keras.regularizers.l2(reg_l2))
                      for i in units]
        self.dropout = keras.layers.Dropout(rate=dropout)
    
    def call(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.dropout(x)
        return x


class SENET(layers.Layer):
    """
    用于计算特征重要性的网络
    """
    def __init__(self, reduction_ratio=8, activation='relu', reg_l2=0.01):
        self.r = reduction_ratio
        self.act = activation
        self.reg_l2 = reg_l2
        super().__init__()
    
    def build(self, input_shape):
        units = [input_shape[-1] // self.r, input_shape[1]]
        self.layers = [keras.layers.Dense(i, activation=self.act, use_bias=False, kernel_regularizer=keras.regularizers.l2(self.reg_l2))
                      for i in units]
        super().build(input_shape)
    
    def call(self, x):
        """
        x：[None, f, k] 
        """
        tmp = tf.identity(x) # copy
        # squeeze
        x = tf.reduce_mean(x, axis=-1) # [None, f]
        print(x.shape)
        # excitation
        for layer in self.layers:
            x = layer(x) # [None f]
        print(x.shape)
        # re-weight
        x = tf.expand_dims(x, axis=-1) # [None, f, 1]
        #
        out = tf.multiply(x, tmp) # [None, f, k]
        return out

class Binlinear(layers.Layer):
    """
    该层主要实现了一个稀疏特征embedding的组合交叉功能
    值得学习的是特征组合的实现方式，使用了组合公式
    """
    def __init__(self, type_='all'):
        assert type_ in ('all', 'each', 'interaction'), '双线性交互层的模式必须是（all, each, interaction）中的一种'
        self.type_ = type_
        self.concat_layer = layers.Concatenate(axis=-1, name='binlinear_concat_layer')
        super().__init__()
    
    def build(self, input_shape):
        """
        input_shape : [None, f, k]
        """
        shape = list(input_shape)
        initializer = keras.initializers.glorot_normal
        if self.type_ == 'all':
            self.wight1 = self.add_weight(shape=(1, shape[-1], shape[-1]),
                                         initializer=initializer,
                                         trainable=True) # [k, k]
        elif self.type_ == 'each':
            self.wight1 = [self.add_weight(shape=(shape[-1], shape[-1]),
                                         initializer=initializer,
                                         trainable=True)
                          for _ in range(shape[1]-1)
                          ]# f-1 * [k, k]
        else:
            self.wight1 = [self.add_weight(shape=(shape[-1], shape[-1]),
                                         initializer=initializer,
                                         trainable=True)
                          for _ in itertools.combinations(range(shape[1]), 2)
                          ] # f*(f-1)/2* [k, k]
        super().build(input_shape)
    
    def call(self, x):
        """
        x: [None, f, k]
        output: [None, f*(f-1)/2, k]
        """
        # 先做内积
        _, f, k = x.shape
        tmp = tf.split(x, f, axis=1) # [(None, 1, k), ...]
        tmp = [tf.squeeze(i, axis=1) for i in tmp] # [(None, k), ...]
        if self.type_ == 'all':
            cdots = [tf.tensordot(item, self.wight1, axes=[-1, 0]) for item in tmp] # f*[(None, k)]
            res = [tf.multiply(
                            cdots[i],
                            tmp[j]) # (None, k)
                  for i, j in itertools.combinations(f, 2) # 得到组合情况
                  ] #f*(f-1)/2* [None, k]
        elif self.type_ == 'each':
            cdots = [tf.tensordot(tmp[i], self.wight1[i], axes=[-1, 0]) for i in range(f-1)] # f-1*[(None, k)]
            res = [tf.multiply(
                            cdots[i],
                            tmp[j])
                  for i, j in itertools.combinations(range(f), 2) # 得到组合情况
                  ] #f*(f-1)/2* [None, k]
        else:
            cdots = [tf.tensordot(tmp[i], self.wight1[idx], axes=[-1, 0]) 
                     for idx, (i, _) in enumerate(itertools.combinations(range(f), 2))] 
            res = [tf.multiply(
                    cdots[idx],
                    tmp[j]
            ) for idx, (_, j) in enumerate(itertools.combinations(range(f), 2))
            ] #f*(f-1)/2* [None, k]
        print('res', tmp[0].shape, cdots[0].shape, res[0].shape)
        out = self.concat_layer(res) #[None, 1, f*(f-1)*k/2]
        return keras.layers.Flatten()(out)

    
class FIBINET(keras.layers.Layer):
    def __init__(self, sparse_feat_dict, feat_columns,
                 dnn_units=(256, 126, 64), dnn_activation='relu', dnn_dropout=0.3,
                senet_reduaction_ratio=0.3,
                 type_='each',
                 embed_reg=0.01,
                ):
        super().__init__()
        self.dnn = DNN(dnn_units, dnn_activation, dnn_dropout)
        self.senet = SENET(senet_reduaction_ratio)
        self.org_binlinear = Binlinear(type_=type_)
        self.senet_binlinear = Binlinear(type_=type_)
        self.concat_layer = keras.layers.Concatenate(name='to_dnn')
        self.last_linear = keras.layers.Dense(1, activation='sigmoid')
        self.bn = keras.layers.BatchNormalization()
        
        self.embed_layers = {
            k: keras.layers.Embedding(v[0], v[1], embeddings_regularizer=keras.regularizers.l2(embed_reg))
            for k, v in sparse_feat_dict.items()
                            }
        self.sparse_user_index, self.sparse_items_index = feat_columns
        
    
    def call(self, inputs):
        dense_user, sparse_user, dense_items, sparse_items = inputs
        
        dense_input = tf.concat([dense_user, dense_items], axis=-1) # [None, k]
        
        print(sparse_user.shape, self.sparse_user_index)
        embed_user = tf.stack(
            [
                self.embed_layers[k](sparse_user[:,idx])
                for k, idx in self.sparse_user_index.items()
            ], axis=1)
         # [None, f_user, user_embeds]
        embed_items = tf.stack(
            [
                self.embed_layers[k](sparse_items[:, idx])
                for k,idx in self.sparse_items_index.items()
            ], axis=1
        ) # [None, f_items, items_embeds]
        sparse_input = tf.concat([embed_user, embed_items], axis=1) #[None, f, embed_dims]
        
        out1 = self.org_binlinear(sparse_input) # [None, f*(f-1)/2*embed_dims]
        out_senet = self.senet(sparse_input) #[None, f, embed_dims]
        out2 = self.senet_binlinear(out_senet) # [None, f*(f-1)/2*embed_dims]
        
        x = self.concat_layer([dense_input, out1, out2]) # [None, tot]
        x = self.bn(x)
        x = self.dnn(x)
        ctr = self.last_linear(x)
        return ctr        
        

In [847]:
pd.set_option('max_columns', 100)
# =============================== GPU ==============================
# gpu = tf.config.experimental.list_physical_devices(device_type='GPU')
# print(gpu)
os.environ['CUDA_VISIBLE_DEVICES'] = '2, 3'
# ========================= Hyper Parameters =======================
# you can modify your file path
file = '../data/criteo_sampled_data.csv'
read_part = True
sample_num = 100000
embed_dim = 10
test_size = 0.2
df = pd.read_csv(file, nrows=4)
df.head()

# ========================== Create dataset =======================
feature_columns, train, test = create_criteo_dataset(
                                    embed_dim=embed_dim, file=file,
                                       read_part=read_part,
                                       sample_num=sample_num,
                                       test_size=test_size)


In [848]:
sparse_feat_dict = {
    item['feat']: (item['feat_num'], item['embed_dim'])
    for item in feature_columns[1]
    if item['feat'].startswith('C')
}


In [849]:
def get_dataset(data):
    (dense, sparse), label = data
    # 模拟数据
    thr = 0.5
    length_dense, length_sparse = dense.shape[1], sparse.shape[1]
    idx_dense, idx_sparse = int(length_dense*thr), int(length_sparse*thr)
    dense_user, dense_items = dense[:, :idx_dense], dense[:,idx_dense:]
    sparse_user, sparse_items = sparse[:,:idx_sparse], sparse[:,idx_sparse:]
    inputs = [dense_user, sparse_user, dense_items, sparse_items]
    print([i.shape[1] for i in inputs])
    sparse_user_index = {f'C{i+1}': i for i in range(idx_sparse)}
    sparse_items_index = {f'C{i+1}': i-idx_sparse for i in range(idx_sparse, length_sparse)}

    return (inputs, label), (sparse_user_index, sparse_items_index)

(train_x, train_y), feature_column_list = get_dataset(train)
(test_x, test_y), _ = get_dataset(test)

[6, 13, 7, 13]
[6, 13, 7, 13]


In [850]:
dense_user_dims = 6
sparse_user_dims = 13
dense_items_dims = 7
sparse_items_dims = 13


def build_model():
    fibinet = FIBINET(sparse_feat_dict, feature_column_list, type_='interaction')
    dense_user_input = keras.Input(shape=(dense_user_dims,), dtype=tf.float32, name='dense_user')
    sparse_user_input = keras.Input(shape=(sparse_user_dims,), dtype=tf.float32, name='sparse_user')
    dense_item_input = keras.Input(shape=(dense_items_dims,), dtype=tf.float32, name='dense_items')
    sparse_items_input = keras.Input(shape=(sparse_items_dims,), dtype=tf.float32, name='sparse_items')
    inputs = [dense_user_input, sparse_user_input, dense_item_input, sparse_items_input]
    outputs = fibinet(inputs)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

keras.backend.clear_session()

model = build_model()
model.compile(loss=keras.losses.binary_crossentropy, 
              optimizer=keras.optimizers.Adam(),
             metrics=[keras.metrics.AUC()])

(None, 13) {'C1': 0, 'C2': 1, 'C3': 2, 'C4': 3, 'C5': 4, 'C6': 5, 'C7': 6, 'C8': 7, 'C9': 8, 'C10': 9, 'C11': 10, 'C12': 11, 'C13': 12}
res (None, 10) (None, 10) (None, 10)
(None, 26)
(None, 26)
res (None, 10) (None, 10) (None, 10)


In [846]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 dense_user (InputLayer)        [(None, 6)]          0           []                               
                                                                                                  
 sparse_user (InputLayer)       [(None, 13)]         0           []                               
                                                                                                  
 dense_items (InputLayer)       [(None, 7)]          0           []                               
                                                                                                  
 sparse_items (InputLayer)      [(None, 13)]         0           []                               
                                                                                              

In [851]:
model.fit(train_x, train_y, epochs=5, batch_size=256, validation_data=(test_x, test_y))

Epoch 1/5
(None, 13) {'C1': 0, 'C2': 1, 'C3': 2, 'C4': 3, 'C5': 4, 'C6': 5, 'C7': 6, 'C8': 7, 'C9': 8, 'C10': 9, 'C11': 10, 'C12': 11, 'C13': 12}
res (None, 10) (None, 10) (None, 10)
(None, 26)
(None, 26)
res (None, 10) (None, 10) (None, 10)
(None, 13) {'C1': 0, 'C2': 1, 'C3': 2, 'C4': 3, 'C5': 4, 'C6': 5, 'C7': 6, 'C8': 7, 'C9': 8, 'C10': 9, 'C11': 10, 'C12': 11, 'C13': 12}
res (None, 10) (None, 10) (None, 10)
(None, 26)
(None, 26)
res (None, 10) (None, 10) (None, 10)
313/313 [==============================] - ETA: 0s - loss: 2.0958 - auc: 0.6514(None, 13) {'C1': 0, 'C2': 1, 'C3': 2, 'C4': 3, 'C5': 4, 'C6': 5, 'C7': 6, 'C8': 7, 'C9': 8, 'C10': 9, 'C11': 10, 'C12': 11, 'C13': 12}
res (None, 10) (None, 10) (None, 10)
(None, 26)
(None, 26)
res (None, 10) (None, 10) (None, 10)
313/313 [==============================] - 154s 344ms/step - loss: 2.0958 - auc: 0.6514 - val_loss: 0.5831 - val_auc: 0.7112
Epoch 2/5
313/313 [==============================] - 86s 274ms/step - loss: 0.5392 - auc: 

In [852]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 dense_user (InputLayer)        [(None, 6)]          0           []                               
                                                                                                  
 sparse_user (InputLayer)       [(None, 13)]         0           []                               
                                                                                                  
 dense_items (InputLayer)       [(None, 7)]          0           []                               
                                                                                                  
 sparse_items (InputLayer)      [(None, 13)]         0           []                               
                                                                                              